Este archivo contiene las funciones que generan las gráficas usadas para el paper del Congreso de Acústica y el informe final de la beca (y más no usadas)

In [ ]:
import import_ipynb
from CIS_6 import *

# Gráficos

In [ ]:
import os
pathCarriers = r"C:/Users/roman/OneDrive/Escritorio/Audio_Code/carriers/"
os.chdir(pathCarriers)

In [ ]:
#pip install standard-aifc Python >= 3.13.0
#pip install standard-sunau  Python >= 3.13.0

In [ ]:
s, _ = librosa.load('clari1.wav', sr = IMPLANT_SR)

## Envolventes Cochlear

In [ ]:
M = 500

envelope = np.zeros(len(s))
if(True):
  for i in range(len(s)): envelope[i] = abs(s[i])

else: #half_wave
  for i in range(len(s)): envelope[i] = max(0., s[i])

#Low - Pass: Butter de 2do orden

b,a = signal.butter(1, 200, 'lowpass', output='ba', fs = IMPLANT_SR)
envelope2 =  signal.filtfilt(b,a, envelope)

sos = signal.butter(1, 200, 'lowpass', fs=IMPLANT_SR, output='sos')
envelope3 = signal.sosfiltfilt(sos, envelope)

b,a = signal.butter(2, 200, 'lowpass', output='ba', fs = IMPLANT_SR)
envelope4 =  signal.lfilter(b,a, envelope)

plt.plot(envelope2[:M], label = 'Envolvente (fase cero)')
plt.plot(envelope4[:M], label = 'Envolvente')
plt.plot(envelope3[:M], label = ' sos')

plt.plot(s[:M], label = 'Señal', alpha = 0.3)
plt.legend(framealpha=1, shadow=True)
plt.grid(alpha=0.25)
plt.show()

In [ ]:
M = 10000
FROM = 11000
TO = 17000
FILTERS_ZERO_PHASE = False

s, _ = librosa.load('clari1.wav', sr = IMPLANT_SR)

outs = filter_bank_CIS(s, GENERIC_COCHLEAR1)
s = s[FROM:TO]
s1 = outs[2][FROM:TO]
plt.plot(s1[:M])

ENVELOPE_ZEROPHASE = False

env1 = envelope_Cochlear(s1[:M], 200, True, gain_compensation = True)
s2 = outs[3][FROM:TO]
plt.plot(s2[:M])

ENVELOPE_ZEROPHASE = False
env2 = envelope_Cochlear(s2[:M], 200, True, gain_compensation = True)

plt.plot(env1)
plt.plot(env2)
display(Audio(s, rate = IMPLANT_SR))
display(Audio(s1+s2, rate = IMPLANT_SR))

In [ ]:
ts = [i/IMPLANT_SR for i in range(len(s2))]
plt.plot(ts, s2[:M], label = 'Señal')
plt.plot(ts, env2, label = 'Envolvente', color = 'r')
plt.xlabel('Tiempo (s)')

plt.legend(framealpha=1, shadow=True)
plt.grid(alpha=0.4)
plt.show()

## N of M

In [ ]:
s, _ = librosa.load('clari1.wav', sr = IMPLANT_SR)
outs = filter_bank_CIS(s, GENERIC_COCHLEAR1)
ts = [i/IMPLANT_SR for i in range(len(s))]

fig, axs = plt.subplots(9,1)
axs[0].plot(ts, s, label = 'Señal de entrada')
axs[0].legend(framealpha=0.8, shadow=True, loc = 'lower right',  prop={'size': 6})

for i in range(1,9):
  envi = envelope_Cochlear(outs[i-1], 200, True, gain_compensation = False)

  axs[i].plot(ts, envi, color = 'black', label = 'Envolvente canal' + str(i))
  axs[i].set_ylim(0, 0.065)

  axs[i].legend(framealpha=0.8, shadow=True, loc = 'upper right',  prop={'size': 6})

  #axs[i].plot(si, label = 'Banda ' + str(i+1), alpha = 0.85)

  #axs[i].legend(framealpha=1, shadow=True, loc = 'lower left')
plt.grid(alpha=0.4)
plt.xlabel('Tiempo (s)')
plt.show()
display(Audio(s, rate = IMPLANT_SR))


Funcion especial para graficar

In [ ]:
def simulate_CIS_ACE_GraficoParalelo(s, elec_array, m):
    filtered_bank = filter_bank_CIS(s, GENERIC_COCHLEAR1)
    envs = [envelope_Cochlear(filtered_bank[i], ENVELOPE_FC, ENVELOPE_FULLWAVE, gain_compensation= False) for i in range(elec_array.Nchann)] # CIS convencional
    n = len(envs[0])
    out = np.zeros(n)
    gs = elec_array.gs

    WIN_SIZE = round(WIN_LEN_ACE * OUT_SR/1000)
    i = 0

    envs_seleccionadas = np.zeros((len(envs), n))
    power = 1

    while(i+WIN_SIZE<n):
      #Extraigo las envolventes de mayor amplitud
      amplitudes_promedio = [sum(np.array(envs[c][i: i+WIN_SIZE])**power) for c in range(elec_array.Nchann)] #Tomo la amplitud promedio en el ciclo de estimulación de cada envolvente
      indices_max = np.argpartition(amplitudes_promedio, -m)[-m:] #Indices de las envolventes de mayor amplitud

      #Estimulo bandas maximas
      for max_channel in indices_max:
        #out[i: i+WIN_SIZE] += filtered_noises[max_channel][i: i+WIN_SIZE] * envs[max_channel][i: i+WIN_SIZE]
        envs_seleccionadas[max_channel][i:i+WIN_SIZE] = envs[max_channel][i:i+WIN_SIZE]

        #Simulo channel interaction:
        if(CHANNEL_INTERACTION_INDEX > 0):
          for other_channel in range(elec_array.Nchann):
            if(other_channel == max_channel): continue
            out[i:i+WIN_SIZE] += filtered_noises[other_channel][i:i+WIN_SIZE] * envs[max_channel][i:i+WIN_SIZE]*gs[max_channel][other_channel]

      i+=WIN_SIZE

    return envs_seleccionadas

Envolventes

In [ ]:
s, _ = librosa.load('clari1.wav', sr = IMPLANT_SR)
n = len(s)#round(WIN_LEN_ACE*IMPLANT_SR/1000*30)
s = s[:n]
outs = filter_bank_CIS(s, GENERIC_COCHLEAR1)
CHANNEL_INTERACTION_INDEX = 0
envsSeleccionadas = simulate_CIS_ACE_GraficoParalelo(s, GENERIC_COCHLEAR1, 3)

CANALES = 8

ts = [i/IMPLANT_SR for i in range(len(s))]

fig, axs = plt.subplots(CANALES+1,1)
axs[0].plot(ts, s, label = 'Señal de entrada')
axs[0].legend(framealpha=0.7, shadow=True, loc = 'lower right',  prop={'size': 6})

for i in range(1,CANALES+1):

  envi = envsSeleccionadas[i-1]

  axs[i].plot(ts, envi, color = 'black', label = 'Envolvente resultante canal' + str(i))
  axs[i].set_ylim(0, 0.08)

  axs[i].legend(framealpha=0.8, shadow=True, loc = 'upper right',  prop={'size': 6})

  #axs[i].plot(si, label = 'Banda ' + str(i+1), alpha = 0.85)

  #axs[i].legend(framealpha=1, shadow=True, loc = 'lower left')
plt.grid(alpha=0.4)
plt.xlabel('Tiempo (s)')
plt.show()
display(Audio(s, rate = IMPLANT_SR))

In [ ]:
GENERIC_MEDEL2b.setearParametros()
IMPLANT_SR = 17000
s, _ = librosa.load('organ.wav', sr = IMPLANT_SR)
s/= np.max(np.abs(s))
outs = filter_bank_CIS(s, GENERIC_MEDEL2b)

fig, axs = plt.subplots(9,2, figsize=(10,12))
axs[0][0].plot(s, label = 'Señal de entrada')
axs[0][0].legend(framealpha=1, shadow=True, loc = 'lower left',  prop={'size': 10})
axs[0][0].get_xaxis().set_visible(False)
ts = [i/IMPLANT_SR for i in range(len(s))]

F = np.abs(fft(s)/len(s))
freqsfft = [IMPLANT_SR/len(s)*i for i in range(len(s)//2) ]
axs[0][1].plot(freqsfft, F[:len(s)//2], color = 'darkcyan', label = 'DFT señal original')
axs[0][1].grid(alpha = 0.5)
axs[0][1].legend(framealpha=1, shadow=True, loc = 'upper right')
axs[0][1].get_xaxis().set_visible(False)

for i in range(1,9):
  si = outs[i]
  Fi = np.abs(fft(si)/len(s))
  axs[i][0].plot(ts, si, color = 'orange', label = 'Canal '+ str(i) + ' ( ' + str(GENERIC_MEDEL2b.freqsAnalisis[i][0]) + 'Hz - ' +str(GENERIC_MEDEL2b.freqsAnalisis[i][1])+ ' Hz)')
  axs[i][0].set_ylim(-1,1)
  axs[i][1].set_ylim(0,0.08)
  axs[i][0].get_xaxis().set_visible(False)
  axs[i][1].get_xaxis().set_visible(False)

  axs[i][1].plot(freqsfft, Fi[:len(s)//2], color = 'fuchsia', label = 'DFT canal ' + str(i))

  print(f'Canal {i}')
  display(Audio(si, rate = IMPLANT_SR))
  #axs[i][1].plot(si, label = 'Banda ' + str(i+1), alpha = 0.85)
  axs[i][0].legend(framealpha=1, shadow=True, loc = 'lower left')
  axs[i][1].legend(framealpha=1, shadow=True, loc = 'lower right')
  axs[i][1].grid(alpha = 0.5)


axs[8][0].set_xlabel('Tiempo (s)')
axs[8][0].get_xaxis().set_visible(True)
axs[8][1].set_xlabel('Freq (Hz)')
axs[8][1].get_xaxis().set_visible(True)


plt.show()

In [ ]:
display(Audio(s,rate= IMPLANT_SR))

s2 = np.zeros(len(s))
for i in range(5,9):
  s2 += outs[i]

display(Audio(s2,rate= IMPLANT_SR))
plt.plot(np.abs(fft(s2)))
plt.show()
plt.plot(np.abs(fft(s)))

## Interleaved

funcion grafico

In [ ]:
def simulate_CIS_Interleaved_Grafico(s, elec_array, m):
  filtered_bank = filter_bank_CIS(s, elec_array)
  envs = [envelope_Cochlear(filtered_bank[i], ENVELOPE_FC, ENVELOPE_FULLWAVE, gain_compensation= False) for i in range(elec_array.Nchann)] # CIS convencional
  n = len(envs[0])

  WIN_SIZE = round(WIN_LEN_ACE * OUT_SR/1000)

  power = 1

  i = 0
  pwnActivo = round(PW*OUT_SR) #samples en cada pulso
  pwnCompleto = round(OUT_SR/m/PPS) #samples activos en cada pulso
  gs = elec_array.gs
  #out = np.zeros(n)
  outs = np.zeros((len(envs), n))

  while(i+pwnCompleto<n):


    #Extraigo las envolventes de mayor amplitud
    #amplitudes_promedio = [sum(envs[c][i:i+pwn]) for c in range(N_chann)] #Tomo la amplitud promedio en el ciclo de estimulación de cada envolvente
    amplitudes_promedio = [sum(np.array(envs[c][i:i+pwnCompleto]**power)) for c in range(elec_array.Nchann)]
    indices_max = np.argpartition(amplitudes_promedio, -m)[-m:] #Indices de las envolventes de mayor amplitud

    for max_channel in indices_max:
      #if(len(interacted_channels_maximos[c][fr: to])<pwn): continue  #LLegué al final de la señal   (todo: filtered_noises?)

      fr = i+max_channel*pwnCompleto
      to = fr + pwnActivo
      #to = i+(max_channel+1)*pwnCompleto
      di = to-fr

      outs[max_channel][fr : to] =  amplitudes_promedio[max_channel]
      outs[max_channel][fr:fr+ di//2] *= -1
      #out[fr: to] = filtered_noises[max_channel][fr: to] * * amplitudes_promedio[max_channel][i//WIN_SIZE]

      #Simulo channel interaction:
      if(CHANNEL_INTERACTION_INDEX>0):
        for other_channel in range(elec_array.Nchann):
          if(other_channel == max_channel): continue
          out[fr:to] += filtered_noises[other_channel][fr:to] * envs[max_channel][fr:to]*gs[max_channel][other_channel]


    i+=m*pwnCompleto

  return outs

In [ ]:
#PARAMETROS Y SEÑAL DE ENTRADA
elec_array = GENERIC_MEDEL3
elec_array.setearParametros()
OUT_SR = 192000
IMPLANT_SR = elec_array.SR
s, _ = librosa.load('clari1.wav', sr = IMPLANT_SR)
CHANNEL_INTERACTION_INDEX = 0

CANALES = 5 #CANALES A MOSTRAR


s/= max(abs(s)) #normalizo
PW = 1/CANALES/PPS/2  #PW maximo = 1/canales/pps. Seteo la mitad del maximo ac
outs1 = simulate_CIS_Interleaved_Grafico(s, elec_array, 5)

PW = 1/CANALES/PPS  #PW maximo = 1/canales/pps. Seteo la mitad del maximo ac
outs2 = simulate_CIS_Interleaved_Grafico(s, elec_array, 5)

outs = [outs1[0], outs1[1], outs1[2], outs2[3], outs2[4]] #primeros tres canales con un PW, ultimos dos con otro

ti = 110 #ms
tf = ti + WIN_LEN_ACE #ms

ts_in = np.array([i/IMPLANT_SR*1000 for i in range(len(s))])
ts_out = np.array([i/OUT_SR*1000 for i in range(len(outs[0]))])

fig, axs = plt.subplots(CANALES+1,1, layout = 'constrained' )
axs[0].plot(ts_in, s, label = 'Señal de entrada')
axs[0].legend(framealpha=0.7, shadow=True, loc = 'lower right',  prop={'size': 6})
maxgraph = np.max(outs)*1.05

for i in range(1,CANALES+1):

  envi = outs[i-1]

  axs[i].plot(ts_out, envi, color = 'black', label = 'Canal ' + str(i), )
  axs[i].set_ylim(-maxgraph, maxgraph )
  #axs[i].get_yaxis().set_visible(True)
  axs[i].set_yticks([], labels = [])
  axs[i].legend(framealpha=0.8, shadow=True,  prop={'size': 6})
  axs[i].set_xlim(ti,tf)
  axs[i].get_xaxis().set_visible(False)
  #axs[i].grid(alpha=0.9)

  #axs[i].plot(si, label = 'Banda ' + str(i+1), alpha = 0.85)

  #axs[i].legend(framealpha=1, shadow=True, loc = 'lower left')
axs[0].set_xlim(ti,ti + WIN_LEN_ACE)
axs[3].set_ylabel('Amplitud', fontsize = 12)
axs[0].get_xaxis().set_visible(False)
axs[-1].get_xaxis().set_visible(True)


plt.xlabel('Tiempo (ms)')
plt.show()
display(Audio(s, rate = IMPLANT_SR))
print('PPS = ' + str(PPS))
print('PW = ' + str(PW*1000) + 'ms = ' + str(PW*OUT_SR) + ' samples')

In [ ]:
print(PW)
print(1/5/PPS)

## Integración

In [ ]:
#Probamos si el algoritmo funciona bien,
#chequeando que en cada momento solo se activen la cantidad especificada de electrodos

#Prueba con habla
#hspeech1 48khz.wav
s, sr = librosa.load('clima (3).wav' , sr = IMPLANT_SR) #Hago el resampleo en este paso
s = s[0:3*sr]
elec_array = GENERIC_MEDEL1
elec_array.setearParametros()
MODE = ACE
CHANNEL_INTERACTION_INDEX = 0
MAXIMA = 2
DISPERTION = 0.001
WIN_LEN_ACE = 8 #ms
#SETEAR_PPS(800)

print('IN')
display(Audio(s, rate = IMPLANT_SR))
plot_stft(s,elec_array, analisis=True, mini= True)

print('OUT')
out = elec_array.CIS(s)
display(Audio(out, rate = OUT_SR))
graph = plot_stft(out,elec_array, analisis=False, mini = True, )

In [ ]:
print(len(s)//IMPLANT_SR)
print(len(out)//OUT_SR)

In [ ]:
#Comprobar espectrograma

In [ ]:
#Probamos si el algoritmo funciona bien,
#chequeando que en cada momento solo se activen la cantidad especificada de electrodos

#Prueba con habla
#hspeech1 48khz.wav
s, sr = librosa.load('clima (3).wav' , sr = IMPLANT_SR) #Hago el resampleo en este paso
s = s[0:3*sr]
elec_array = GENERIC_MEDEL1
elec_array.setearParametros()
MODE = ACE
CHANNEL_INTERACTION_INDEX = 0
MAXIMA = 1
DISPERTION = 0
WIN_LEN_ACE = 4 #ms
#SETEAR_PPS(800)

print('IN')
display(Audio(s, rate = IMPLANT_SR))
plot_stft(s,elec_array, analisis=True, mini= True)

print('OUT')
out = elec_array.CIS(s)
display(Audio(out, rate = OUT_SR))
graph = plot_stft(out,elec_array, analisis=False, mini = True )

In [ ]:
pares = GENERIC_COCHLEAR1.freqsAnalisis
for par in pares:
  print('(' + str(round(par[0])) + ' - ' + str(round(par[1]))+')')



In [ ]:
#Implante
elec_array = GENERIC_MEDEL1
MODE = CONTINOUS
DISPERTION = 0
CHANNEL_INTERACTION_INDEX = 0

#Señal
freq = 409
s = np.array(puretone(freq, 3, IMPLANT_SR))*np.array(puretone(0.5, 3, IMPLANT_SR))  #envolvente lenta
print('Audio original')
display(Audio(s, rate = IMPLANT_SR))
plot_stft(s, elec_array, analisis = True, mini = True)

#Procesamiento
out = elec_array.CIS(s)
outfile = 'Out: Generic MEDEL2 - CIS CONTINOUS PURETONE ' + str(freq) + 'hz.wav'
wavfile.write(outfile, OUT_SR, out)

print(outfile)
display(Audio(out, rate = OUT_SR))
plot_stft(out,elec_array, analisis = False)

#Todo: Logear los parámetros usados

In [ ]:
#Implante
elec_array = GENERIC_MEDEL1
MODE = ACE
DISPERTION = 1
CHANNEL_INTERACTION_INDEX = 0
MAXIMA = 1
ORDEN = 6

freq = 2409
s = np.array(puretone(freq, 3, IMPLANT_SR))*np.array(puretone(0.5, 3, IMPLANT_SR))  #envolvente lenta

#Procesamiento
out = elec_array.CIS(s)

outfile = 'Out: Generic MEDEL2 - CIS CONTINOUS PURETONE ' + str(freq) + 'hz.wav'
wavfile.write(outfile, OUT_SR, out)

print(outfile)
display(Audio(out, rate = OUT_SR))
plot_stft(out,elec_array, analisis = False, mini = True)

#Todo: Logear los parámetros usados

In [ ]:
#Implante
elec_array = GENERIC_MEDEL1
MODE = ACE
DISPERTION = 1
CHANNEL_INTERACTION_INDEX = 0
MAXIMA = 1
ORDEN = 6

freq = 2409
import scipy
t = [i/IMPLANT_SR for i in range(4*IMPLANT_SR)]
s =  scipy.signal.chirp(t, f0=20, f1=IMPLANT_SR/2, t1=4, method='linear') #chirp

#freq = 409
print('Audio original')
display(Audio(s, rate = IMPLANT_SR))
plot_stft(s, elec_array, analisis = True, mini = True)

#Procesamiento
out = elec_array.CIS(s)

outfile = 'Out: Generic MEDEL2 - CIS CONTINOUS PURETONE ' + str(freq) + 'hz.wav'
wavfile.write(outfile, OUT_SR, out)

print(outfile)
display(Audio(out, rate = OUT_SR))
plot_stft(out,elec_array, analisis = False, mini = True)

#Todo: Logear los parámetros usados

In [ ]:
#Implante
elec_array = GENERIC_MEDEL1
MODE = ACE
DISPERTION = 1
CHANNEL_INTERACTION_INDEX = 0
MAXIMA = 4
ORDEN = 6

freq = 2409
import scipy
t = [i/IMPLANT_SR for i in range(4*IMPLANT_SR)]
s =  scipy.signal.chirp(t, f0=20, f1=IMPLANT_SR/2, t1=4, method='linear') #chirp

#freq = 409
print('Audio original')
display(Audio(s, rate = IMPLANT_SR))
plot_stft(s, elec_array, analisis = True, mini = True)

#Procesamiento
out = elec_array.CIS(s)

outfile = 'Out: Generic MEDEL2 - CIS CONTINOUS PURETONE ' + str(freq) + 'hz.wav'
wavfile.write(outfile, OUT_SR, out)

print(outfile)
display(Audio(out, rate = OUT_SR))
plot_stft(out,elec_array, analisis = False, mini = True)

#Todo: Logear los parámetros usados

In [ ]:
ramp = [i/1000 for i in range(1100)]
out = [compress_envelope_sample(ramp[i]) for i in range(120)]
plt.plot(out)

In [ ]:
compress_envelope_sample(1)

In [ ]:
10**((100-T_SPL)/20): sample = 0
  if sample > 10**((C_SPL)/20):

In [ ]:
10**(-(100-C_SPL)/20)

-------------------------------------

# Paper

In [ ]:
import os
os.chdir(r'C:\Users\roman\OneDrive\Escritorio\Cochlear Implants y Tesis MyT\CODE ultima version')
%run ./CIS_6.ipynb
os.chdir(r'C:\Users\roman\Desktop\Audio_Code\carriers')

#Disclaimer esto no me anduvo mas..

In [ ]:
freqs = GENERIC_MEDEL1.freqsAnalisis
s = puretone(freqs[5],3,OUT_SR)

for f in range(len(freqs)):
   s += puretone(freqs[f], 3, OUT_SR)*GENERIC_MEDEL1.gs[5][f]

display(Audio(s, rate  = OUT_SR))
plot_fft(s, GENERIC_MEDEL1, f_from = 100, f_to = 5000, range_dB = 60)

In [ ]:
CHANNEL_INTERACTION_INDEX  = 2
ASYMMETRY_INDEX = 0
DISPERTION = 0
dur = 0.1
order = 6
elec_array = GENERIC_MEDEL1
elec_array.setearParametros()
A = 1

def inter(A,g):
   CII = CHANNEL_INTERACTION_INDEX
   return CII* ( (1+A) ** g*CII)

freqs = elec_array.freqsAnalisis
s = A*filtered_noise(freqs[5], freqs[5]/10,order,dur,OUT_SR)

for f in range(len(freqs)):
   s += filtered_noise(freqs[f], freqs[f]/10,order,dur,OUT_SR)*inter(A,elec_array.gs[5][f])

display(Audio(s, rate  = OUT_SR))
plot_fft(s, elec_array, f_from = 100, f_to = 5000, range_dB = 60)

In [ ]:
s = puretone(600, 3, 44100)
x = [i/44100 for i in range(3*44100)]
plot_fft(s, GENERIC_MEDEL1, 100, 2000, showElecs=True)

---------------

Demostración de la interacción

A:  sin interaccion

In [ ]:
CHANNEL_INTERACTION_INDEX  = 0
ASYMMETRY_INDEX = 1
DISPERTION = 0
dur = 0.1
order = 6
elec_array = GENERIC_MEDEL1
elec_array.setearParametros()
A = 1

def inter(A,g):
   return A*CHANNEL_INTERACTION_INDEX*g

freqs = elec_array.freqsElectrodos
s_orig = A*filtered_noise(freqs[5], freqs[5]/10,order,dur,OUT_SR)*80
s_interacted = np.array( [ filtered_noise(freqs[f], freqs[f]/10,order,dur,OUT_SR) for f in range(len(freqs)) ] )
s = s_orig.copy()

for f in range(len(freqs)):
   s += s_interacted[f]*inter(A,elec_array.gs[5][f])

display(Audio(s, rate  = OUT_SR))
fig, ax = plot_fft(s, elec_array, f_from = 100, f_to = 12000, range_dB = 50, showElecs = False)
fig, ax = plot_fft(s, elec_array, f_from = 100, f_to = 12000, range_dB = 50, showElecs = True)

B: Con interaccion

In [ ]:
CHANNEL_INTERACTION_INDEX  = 30
ASYMMETRY_INDEX = 0
elec_array.setearParametros()

s = s_orig.copy()

for f in range(len(freqs)):
   s += s_interacted[f]*inter(A,elec_array.gs[5][f])

display(Audio(s, rate  = OUT_SR))
fig, ax = plot_fft(s, elec_array, f_from = 100, f_to = 12000, range_dB = 60)
fig, ax = plot_fft(s, elec_array, f_from = 100, f_to = 12000, range_dB = 60, showElecs= True)

C: Con asimetria

In [ ]:
CHANNEL_INTERACTION_INDEX  = 30
ASYMMETRY_INDEX = 0.6
elec_array.setearParametros()

s = s_orig.copy()

for f in range(len(freqs)):
   s += s_interacted[f]*inter(A,elec_array.gs[5][f])

display(Audio(s, rate  = OUT_SR))
fig, ax = plot_fft(s, elec_array, f_from = 100, f_to = 12000, range_dB = 60)
fig, ax = plot_fft(s, elec_array, f_from = 100, f_to = 12000, range_dB = 60, showElecs=True)

Ahora lo mismo, pero con el output real del sim

In [ ]:
CHANNEL_INTERACTION_INDEX  = 0.4
ASYMMETRY_INDEX = 0
DISPERTION = 0.4
MODE = ACE
MAXIMA = 1
dur = 0.1
order = 6
elec_array = GENERIC_MEDEL1
elec_array.setearParametros()
A = 1

freqs = elec_array.freqsAnalisis
s = puretone(freqs[5], dur, IMPLANT_SR)
out1 = elec_array.CIS(s)*100

#display(Audio(out, rate  = OUT_SR))
#fig, ax = plot_fft(out, elec_array, f_from = 100, f_to = 12000, range_dB = 50, showElecs = False)
#fig, ax = plot_fft(out, elec_array, f_from = 100, f_to = 12000, range_dB = 50, showElecs = True)



############################################################

ASYMMETRY_INDEX = 0.8
out2 = elec_array.CIS(s)*100


##########################################################


from scipy.fft import rfft, rfftfreq


f_from = 100
f_to = 10000
range_dB = 50
norm = 1
logx = True

sr = OUT_SR
T = 1/sr #periodo
N = len(out1)
t = np.arange(0, N*T, T) # Sampling times

# FFT and bin centers
f = rfftfreq(N, T)

y1 = rfft(out1)
y1 = np.abs(y1)*2/N
y1 /= norm
#y = 20*np.log10(np.abs(y)) #dB
y_db1 = 20*np.log10(y1)

y2 = rfft(out2)
y2 = np.abs(y2)*2/N
y2 /= norm
#y = 20*np.log10(np.abs(y)) #dB
y_db2 = 20*np.log10(y2)


#df : sr/N  --> f(i) = i* sr/N --> i = f*N/sr
if(f_to == 0): fto = sr/2
ifrom = round(f_from*N/sr)
ito = round(f_to*N/sr)

# Plot spectrum
fig, ax = plt.subplots(dpi = 200)
if(logx): ax.set_xscale('log')

ax.plot(f[ifrom:ito], y_db1[ifrom:ito], color = 'black', linewidth = 0.5) #eje lineal, y logaritmico
ax.plot(f[ifrom:ito], y_db2[ifrom:ito], color = 'red', linewidth = 0.5) #eje lineal, y logaritmico
y_max =  max(y_db1) +5
y_min = max(y_db1) -range_dB
ax.set_ylim(y_min , y_max)

ax.legend((f'I = {CHANNEL_INTERACTION_INDEX}, A = 0', f'I = {CHANNEL_INTERACTION_INDEX}, A = {ASYMMETRY_INDEX}'), loc="upper right")

showElecs = True
if(showElecs):

    color = 'blue'
    secax = ax.secondary_xaxis('top') #eje secundario arriba
    secax.set_xlabel('(Frecuencia característica de los electrodos)', color = 'dimgray', style = 'italic')


    #Creo marquitas para los elec
    elec_labels = ['$e_{' +str(i) +'}$ ' for i in range(elec_array.Nchann) ]

    #Agrego marquitas
    step =1
    secax.set_xticks(elec_array.freqsElectrodos[::step], labels = elec_labels[::step])

    #Les pongo color
    for i in range(1,elec_array.Nchann//step+1):
      secax.get_xticklabels()[-i].set_color('limegreen')
      secax.get_xticklabels()[-i].set_rotation(45)
      secax.get_xticklabels()[-i].set_rotation_mode('anchor')


    ax.vlines(elec_array.freqsElectrodos,y_min , y_max , colors=color, linewidth = 1.3, linestyle = (0, (1,5)), alpha = 0.5)

plt.xlabel('Frecuencia (Hz)')
plt.ylabel('Amplitud (dB)')
plt.grid(alpha  = 0.8)


--------------------------------------------------------------------

Demostración del ancho de banda

In [ ]:
CHANNEL_INTERACTION_INDEX  = 0
ASYMMETRY_INDEX = 0
DISPERTION = 0.2
MAXIMA = 1
dur = 0.1
order = 6
elec_array = GENERIC_MEDEL1
A = 60

freqs = elec_array.freqsAnalisis
#s = A*filtered_noise(freqs[5], freqs[5]/10,order,dur,OUT_SR)
s = A*puretone(freqs[5], dur, IMPLANT_SR)
out1 = elec_array.CIS(s)

############################################################

DISPERTION = 0.8
out2 = elec_array.CIS(s)


##########################################################


from scipy.fft import rfft, rfftfreq


f_from = 100
f_to = 10000
range_dB = 50
norm = 1
logx = True

sr = OUT_SR
T = 1/sr #periodo
N = len(out1)
t = np.arange(0, N*T, T) # Sampling times

# FFT and bin centers
f = rfftfreq(N, T)

y1 = rfft(out1)
y1 = np.abs(y1)*2/N
y1 /= norm
#y = 20*np.log10(np.abs(y)) #dB
y_db1 = 20*np.log10(y1)

y2 = rfft(out2)
y2 = np.abs(y2)*2/N
y2 /= norm
#y = 20*np.log10(np.abs(y)) #dB
y_db2 = 20*np.log10(y2)


#df : sr/N  --> f(i) = i* sr/N --> i = f*N/sr
if(f_to == 0): fto = sr/2
ifrom = round(f_from*N/sr)
ito = round(f_to*N/sr)

# Plot spectrum
fig, ax = plt.subplots(dpi = 200)
if(logx): ax.set_xscale('log')

ax.plot(f[ifrom:ito], y_db1[ifrom:ito], color = 'black', linewidth = 0.5) #eje lineal, y logaritmico
ax.plot(f[ifrom:ito], y_db2[ifrom:ito], color = 'red', linewidth = 0.5) #eje lineal, y logaritmico
y_max =  max(y_db1) +5
y_min = max(y_db1) -range_dB
ax.set_ylim(y_min , y_max)

ax.legend(('D = 0.2', 'D = 0.8'), loc="upper right")


plt.xlabel('Frecuencia (Hz)')
plt.ylabel('Amplitud (dB)')
plt.grid()




In [ ]:
fig, ax = plot_fft(out1, elec_array, f_from = 100, f_to = 10000, range_dB = 60)

Cantidad de canales

In [ ]:
#Probamos si el algoritmo funciona bien,
#chequeando que en cada momento solo se activen la cantidad especificada de electrodos
MODE = ACE
CHANNEL_INTERACTION_INDEX = 0
MAXIMA = 2
DISPERTION = 0.1
WIN_LEN_ACE = 4 #ms
WIN_LEN_AUTO = False
CDL_OC = 30
EXP_OYENTE = False
elec_array = GENERIC_COCHLEAR1
elec_array.setearParametros()

#Prueba con habla
#hspeech1 48khz.wav
i = np.random.randint(3900)
s, sr = librosa.load(f'es_ar_female/ar_f ({i}).wav' , sr = IMPLANT_SR) #Hago el resampleo en este paso

print('IN')
display(Audio(s, rate = IMPLANT_SR))

print('OUT C')
out = elec_array.CIS(s)
display(Audio(out, rate = OUT_SR))
plot_stft(out,elec_array, analisis=False, top_db = 20)



elec_array = GENERIC_MEDEL3
elec_array.setearParametros()
print('OUT M')
out = elec_array.CIS(s)
display(Audio(out, rate = OUT_SR))
plot_stft(out,elec_array, analisis=False, top_db = 20)

In [ ]:
#Probamos si el algoritmo funciona bien,
#chequeando que en cada momento solo se activen la cantidad especificada de electrodos
MODE = ACE
CHANNEL_INTERACTION_INDEX = 0
MAXIMA = 1
DISPERTION = 0.8
WIN_LEN_ACE = 4 #ms
WIN_LEN_AUTO = False
CDL_OC = 30
EXP_OYENTE = False
elec_array = GENERIC_COCHLEAR1
elec_array.setearParametros()

#Prueba con habla
#hspeech1 48khz.wav
i = np.random.randint(3900)
s = puretone(elec_array.freqsAnalisis[4], 3, IMPLANT_SR)

print('IN')
display(Audio(s, rate = IMPLANT_SR))

print('OUT C')
out = elec_array.CIS(s)
display(Audio(out, rate = OUT_SR))
plot_stft(out,elec_array, analisis=False, top_db = 20)



elec_array = GENERIC_MEDEL3
elec_array.setearParametros()
print('OUT M')
out = elec_array.CIS(s)
display(Audio(out, rate = OUT_SR))
plot_stft(out,elec_array, analisis=False, top_db = 20)

Frecuencias de los electrodos

In [ ]:
GENERIC_MEDEL3 = Elec_Array(N=12, xi = 28, dx = 23.1/11) #El de Francisco, Flex28




import matplotlib.ticker as ticker
# Plot spectrum
fig, ax = plt.subplots(dpi = 150) #default dpi is 100
ax.set_xscale('log')
ax.set_xlabel('Frecuencia (Hz)', labelpad = 20)
ax.xaxis.set_tick_params(pad = 18)
ax.set_xlim(150 , 15000)
ax.set_ylim(-60 , 0)
ax.set_yticks([])
ax.set_xticks([])


#####################################
elec_array = GENERIC_MEDEL3

MISMATCH = 0
CDL_OC = 35

elec_array.setearParametros()
freqs = elec_array.freqsElectrodos

color = 'blue'
secax = ax.secondary_xaxis('top') #eje secundario arriba
secax.set_xlabel('Frecuencias características calculadas para los electrodos', color = 'k', style = 'italic', labelpad = 20)

#Creo marquitas para los elec
elec_labels = ['$e_{' +str(i) +'}$ ' for i in range(elec_array.Nchann) ]

#Creo texto con las freqs
secax_freqs = ax.secondary_xaxis('top') #eje secundario arriba
freq_labels = [f'${round(freq)}$ ' for freq in freqs ]


#Agrego marquitas
step = 1
secax.set_xticks(elec_array.freqsElectrodos[::step], labels = elec_labels[::step])
secax.xaxis.set_tick_params(pad = 0)
secax_freqs.set_xticks(freqs, labels = freq_labels[::step])
secax_freqs.xaxis.set_tick_params(pad = 11, labelcolor = color, labelsize= 7)


#Les pongo color
for i in range(1,elec_array.Nchann//step+1):
    secax.get_xticklabels()[-i].set_color(color)
    secax.get_xticklabels()[-i].set_rotation(45)
    secax.get_xticklabels()[-i].set_rotation_mode('anchor')


ax.vlines(elec_array.freqsElectrodos,-40 , 0 , colors=color, linewidth = 1.5, linestyle = 'solid', alpha = 0.85)
#####################################################

MISMATCH = 0.5
CDL_OC = 37

elec_array.setearParametros()
freqs = elec_array.freqsElectrodos

color = 'red'
secax = ax.secondary_xaxis('bottom') #eje secundario arriba

#Creo marquitas para los elec
elec_labels = ['$e_{' +str(i) +'}$ ' for i in range(elec_array.Nchann) ]

#Creo texto con las freqs
secax_freqs = ax.secondary_xaxis('bottom') #eje secundario arriba
freq_labels = [f'${round(freq)}$ ' for freq in freqs ]

#Agrego marquitas
step = 1
secax.set_xticks(freqs[::step], labels = elec_labels[::step])
secax.xaxis.set_tick_params(pad = 0)
secax_freqs.set_xticks(freqs, labels = freq_labels[::step])
secax_freqs.xaxis.set_tick_params(pad = 11, labelcolor = color, labelsize= 7)


#Les pongo color
for i in range(elec_array.Nchann):
    secax.get_xticklabels()[i].set_color(color)
    secax.get_xticklabels()[i].set_rotation(45)
    secax.get_xticklabels()[i].set_rotation_mode('anchor')


ax.vlines(elec_array.freqsElectrodos,-60 , -20 , colors=color, linewidth = 1.5, linestyle = 'solid', alpha = 0.85)

############################################

plt.grid(alpha  = 0.8)
plt.show()



Otra versión

In [ ]:
#####################################
GENERIC_MEDEL3 = Elec_Array(N=12, xi = 28, dx = 23.1/11) #El de Francisco, Flex28
elec_array = GENERIC_MEDEL3

MISMATCH = 0
CDL_OC = 35
elec_array.setearParametros()
freqs1 = elec_array.freqsElectrodos
y1 = [ 0.6 for f in freqs1]

MISMATCH = 0.5
CDL_OC = 37
elec_array.setearParametros()
freqs2 = elec_array.freqsElectrodos
y2 = [ 0.4 for f in freqs2]

###########

import matplotlib.ticker as ticker
xmin = 220
xmax = 13000

fig, ax = plt.subplots(figsize=(16,6), dpi= 130)
ax.set_xscale('log')
ax.set_xlabel('Frecuencia (Hz)', labelpad = 15,fontdict={'size':25} )
ax.xaxis.set_tick_params(pad = 18)
ax.set_xlim(xmin , xmax)
ax.set_ylim(0 , 1)
ax.set_yticks([])
ax.set_xticks([])
ax.hlines(y=[0.4,0.6], xmin=xmin, xmax=xmax, color='gray', alpha=0.85, linewidth=1, linestyles='dashdot')

#cambiar esto a Title
ax.set_title('Frecuencias características calculadas para cada electrodo', color = 'k', style = 'italic', pad = 20, fontdict={'size':32, 'font': 'Times New Roman'}, )

#Creo marquitas para los elec
elec_labels = ['$e_{' +str(i) +'}$ ' for i in range(elec_array.Nchann) ]

#############################################3



#Creo texto con las freqs
secax = ax.secondary_xaxis('top') #eje secundario arriba (e_1, e_2, ...)
secax_freqs = ax.secondary_xaxis('top') #eje secundario arriba    (100hz, 200hz, ...)
freq_labels = [f'${round(freq)}$ ' for freq in freqs1 ]


#Elecs de arriba
color = 'red'
step = 1
secax.set_xticks(freqs1[::step], labels = elec_labels[::step])
secax.xaxis.set_tick_params(pad = 25, labelcolor = color, labelsize = 20, rotation = 30)
secax_freqs.set_xticks(freqs1, labels = freq_labels[::step])
secax_freqs.xaxis.set_tick_params(pad = 6, labelcolor = color, labelsize= 20)
ax.scatter(y=y1, x=freqs1, s=75, color=color, alpha=0.85, label = 'MM = 0 mm,    CDL = 35 mm') #Los puntitos
ax.grid(alpha = 0.7)

#Les pongo color
for i in range(1,elec_array.Nchann//step+1):
    secax.get_xticklabels()[-i].set_rotation_mode('anchor')

#############################


#Creo texto con las freqs
secax_freqs = ax.secondary_xaxis('bottom') #eje secundario arriba
freq_labels = [f'${round(freq)}$ ' for freq in freqs2 ]


#Elecs de abajo
color = 'blue'
step = 1
ax.set_xticks(freqs2[::step], labels = elec_labels[::step])
ax.xaxis.set_tick_params(pad = 5, labelcolor = color, labelsize = 20, rotation = 30)
secax_freqs.set_xticks(freqs2, labels = freq_labels[::step])
secax_freqs.xaxis.set_tick_params(pad = 25, labelcolor = color, labelsize= 20)
ax.scatter(y=y2, x=freqs2, s=75, color=color, alpha=0.85, label = 'MM = 0.5 mm, CDL = 37 mm' ) #Los puntitos
ax.grid(alpha = 0.7)

ax.set_xticks([], minor=True)

#Les pongo color
for i in range(1,elec_array.Nchann//step+1):
    ax.get_xticklabels()[-i].set_rotation_mode('anchor')




######################################

# Title, Label, Ticks and Ylim
#ax.set_yticks(df.index)
#ax.set_yticklabels(df.manufacturer.str.title(), fontdict={'horizontalalignment': 'right'})
plt.legend(framealpha=1, shadow=True,  loc = 'upper right',  prop={'size': 20})
plt.show()

-----------------------------------

Cantidad de canales

In [ ]:
dur = 3
sr = IMPLANT_SR
from numpy import sin, pi
def fmTone(fc, fm, I):
  return  sin( [2*pi*fc*i/IMPLANT_SR + I*fc*sin(2*pi*fm*i/IMPLANT_SR) for i in range(round(3*IMPLANT_SR)) ] )

In [ ]:
#Probamos si el algoritmo funciona bien,
#chequeando que en cada momento solo se activen la cantidad especificada de electrodos
MODE = ACE
CHANNEL_INTERACTION_INDEX = 0
MAXIMA = 1
DISPERTION = 0.05
WIN_LEN_ACE = 4 #ms
WIN_LEN_AUTO = False
CDL_OC = 35
EXP_OYENTE = True
elec_array = GENERIC_MEDEL3
elec_array.setearParametros()

#Prueba con habla
#hspeech1 48khz.wav

s1 = fmTone(500, 1, 0.2)*0.25
s2 = fmTone(2000, 1, 0.2)*0.5
s3 = fmTone(5000, 1, 0.2)*0.8
s = (s1+s2+s3) / 3

print('IN')
display(Audio(s, rate = IMPLANT_SR))
plot_stft(s,elec_array, analisis=True, top_db = 20, mostrarLineas=False)

print(f'OUT M = {MAXIMA}')
out1 = elec_array.CIS(s)
display(Audio(out1, rate = OUT_SR))
plot_stft(out1,elec_array, analisis=False, top_db = 20, mostrarLineas=False)

MAXIMA = 2
elec_array.setearParametros()
print(f'OUT M = {MAXIMA}')
out2 = elec_array.CIS(s)
display(Audio(out2, rate = OUT_SR))
plot_stft(out2,elec_array, analisis=False, top_db = 20, mostrarLineas=False)

Mejor versión

In [ ]:
#Grafico
fig, ax = plt.subplots(3, 1, figsize = (15,11), dpi = 175, sharex= True)
#ax.set_ylim([0, 1])
S = [s, out1, out2]
for i  in range(3):
  x = S[i]
  x = np.array(x)
  n_fft = 512
  win_length = n_fft
  hop_length = n_fft//4
  top_db = 30

  #Espectro
  x_stft = np.abs(librosa.stft(x, n_fft=n_fft, win_length = win_length, hop_length = hop_length)) #hop = n_fft//4
  x_stft_db = librosa.amplitude_to_db(x_stft, ref=np.max, top_db = top_db)
  n_frames = len(x_stft_db[0])

  #Seteo samplig rate según si es señal de entrada al implante o salida del simulador
  if i == 0: sampling_rate = IMPLANT_SR
  else:
    sampling_rate = OUT_SR

  cmap = 'magma'
  img = librosa.display.specshow(data=x_stft_db, y_axis='log', x_axis = 'time',ax=ax[i],
                            sr=sampling_rate, cmap= cmap,
                            n_fft=n_fft, win_length = win_length, hop_length = hop_length)

 # fig.colorbar(img, ax= ax[i], format='%+2.0f dB')
  title = ''
  if( i== 0): title = 'Señal de entrada'
  if (i ==1): title = 'Frecuencia de los electrodos activados (M = 1)'
  if (i ==2): title = 'Frecuencia de los electrodos activados (M = 2)'

  ax[i].set_title('Espectrograma: ' + title, fontdict = {'size'   : 22} ) #'weight' : 'bold'
  ax[i].label_outer()
  ax[i].set_ylim(256,min(14000, sampling_rate/2))
  ax[i].set_ylabel("Frecuencia (Hz)", size = 20)
  for tick in ax[i].yaxis.get_major_ticks():
      tick.label.set_fontsize(16)

fig.colorbar(img, ax=ax,  format='%+2.0f dB')
ax[2].set_xlabel("Tiempo (s)", size = 20)
for tick in ax[2].xaxis.get_major_ticks():
    tick.label.set_fontsize(14)
plt.show()

In [ ]:
s = np.zeros(10000)
s[1000] = 1
s[1001] = -1
s[1002] = 1
s[1003] = -1

Audio(s, rate = 10000)


In [ ]:
s = np.zeros(10000)
s[1000] = 1
s[1001] = 0
s[1002] = -1
s[1003] = 0

Audio(s, rate = 10000)

## Vibrato

In [ ]:
#Probamos si el algoritmo funciona bien,
#chequeando que en cada momento solo se activen la cantidad especificada de electrodos
MODE = ACE
CHANNEL_INTERACTION_INDEX = 0
MAXIMA = 1
DISPERTION = 0.05
WIN_LEN_ACE = 4 #ms
WIN_LEN_AUTO = False
CDL_OC = 35
EXP_OYENTE = True
elec_array = GENERIC_COCHLEAR1
elec_array.setearParametros()

freqs = elec_array.freqsAnalisis
freqs2 =   elec_array.freqsElectrodos

f0 = freqs[5][0]/2 + freqs[5][1]/2

s0 = puretone(f0, 2, IMPLANT_SR)
s = fmTone(f0, 2, 0.005)


plot_stft(s, elec_array, analisis = True)

print('orig')
display(Audio(s0, rate = IMPLANT_SR))



print('FM')
display(Audio(s, rate = IMPLANT_SR))

Centro de un canal

In [ ]:
#Probamos si el algoritmo funciona bien,
#chequeando que en cada momento solo se activen la cantidad especificada de electrodos
MODE = ACE
CHANNEL_INTERACTION_INDEX = 0
MAXIMA = 1
DISPERTION = 0.05
WIN_LEN_ACE = 4 #ms
WIN_LEN_AUTO = False
CDL_OC = 35
EXP_OYENTE = True
elec_array = GENERIC_COCHLEAR1
elec_array.setearParametros()

freqs = elec_array.freqsAnalisis
freqs2 =   elec_array.freqsElectrodos

f0 = freqs[5][0]/2 + freqs[5][1]/2

s0 = puretone(f0, 2, IMPLANT_SR)
ss = []

for i in range(1,10):
    ss.append(fmTone(f0, 2, 0.004*i))


plot_stft(s, elec_array, analisis = True)

print('orig')
display(Audio(s0, rate = IMPLANT_SR))

for i in range(1,10):
    print(f'FM {i}')
    display(Audio(ss[i-1], rate = IMPLANT_SR))

Entre dos canales

In [ ]:
#Probamos si el algoritmo funciona bien,
#chequeando que en cada momento solo se activen la cantidad especificada de electrodos
MODE = ACE
CHANNEL_INTERACTION_INDEX = 0
MAXIMA = 1
DISPERTION = 0.05
WIN_LEN_ACE = 4 #ms
WIN_LEN_AUTO = False
CDL_OC = 35
EXP_OYENTE = True
elec_array = GENERIC_COCHLEAR1
elec_array.setearParametros()

freqs = elec_array.freqsAnalisis
freqs2 =   elec_array.freqsElectrodos

f1 = freqs[5][0]/2 + freqs[5][1]/2
f2 = freqs[6][0]/2 + freqs[6][1]/2
f0 = f1/2 + f2/2
f0 = f1

s0 = puretone(f0, 2, IMPLANT_SR)
ss = []

for i in range(1,5):
    ss.append(fmTone(f0, 2, 0.004*i))



plot_stft(s, elec_array, analisis = True)

print('orig')
display(Audio(s0, rate = IMPLANT_SR))

for i in range(1,5):
    print(f'FM {i}')
    display(Audio(ss[i-1], rate = IMPLANT_SR))
    print('Out')
    out = elec_array.CIS(ss[i-1])
    display(Audio(out, rate = OUT_SR))
    plot_stft(out, elec_array, analisis = False)


In [ ]:
#Probamos si el algoritmo funciona bien,
#chequeando que en cada momento solo se activen la cantidad especificada de electrodos
MODE = ACE
CHANNEL_INTERACTION_INDEX = 0
MAXIMA = 3
DISPERTION = 0.05
WIN_LEN_ACE = 4 #ms
WIN_LEN_AUTO = False
CDL_OC = 35
EXP_OYENTE = True
elec_array = GENERIC_COCHLEAR1
elec_array.setearParametros()

freqs = elec_array.freqsAnalisis
freqs2 =   elec_array.freqsElectrodos

f1 = freqs[5][0]/2 + freqs[5][1]/2
f2 = freqs[6][0]/2 + freqs[6][1]/2
f0 = f1/2 + f2/2

s0 = puretone(f0, 2, IMPLANT_SR)
ss = []

for i in range(1,10):
    ss.append(fmTone(f0, 2, 0.004*i))



plot_stft(s, elec_array, analisis = True)

print('orig')
display(Audio(s0, rate = IMPLANT_SR))

for i in range(1,10):
    print(f'FM {i}')
    display(Audio(ss[i-1], rate = IMPLANT_SR))
    print('Out')
    out = elec_array.CIS(ss[i-1])
    display(Audio(out, rate = OUT_SR))
    plot_stft(out, elec_array, analisis = False)


# Tesis

In [ ]:
#pip install import-ipynb
import import_ipynb
#import CIS_6
from CIS_6 import *

In [ ]:
GENERIC_MEDEL2b.setearParametros()
IMPLANT_SR = 17000

## Banco de filtros H(S)

In [ ]:
fig, ax = plt.subplots(figsize = (12, 7), layout='constrained')
IMPLANT_SR = 17000
GENERIC_MEDEL2b.setearParametros()


s = [1] + [0] * 4095 #delta
outs = filter_bank_analisis(s, GENERIC_MEDEL2b, 6)

for out in outs:
    x = out
    sr = 17000
    fs = np.linspace(0,sr/2, len(x)//2)
    #F = np.abs(fft(x)[:len(x)//2]) / len(x)*2
    F = np.abs(fft(x)[:len(x)//2])
    ax.plot(fs, F)


ax.set_xscale('log')
ax.set_xlim(40)
ax.set_xlabel('Frecuencia $f$ (Hz)', fontsize = 20, font = 'Times New Roman')
ax.set_ylabel('$| H(f) |$', fontsize = 20, font = 'Times New Roman')

ax.set_xticks([50, 100, 200, 500, 1000, 2000, 5000, 8000])
ax.tick_params(axis='both', which='major', labelsize=13)

ax.text
#ax.hlines(1/np.sqrt(2), 0, 8000)  #puntos de interseccion

from matplotlib.ticker import ScalarFormatter
#Hago que los numeros del eje x sean mas lindos
for axis in [ax.xaxis, ax.yaxis]:
    formatter = ScalarFormatter()
    formatter.set_scientific(False)
    axis.set_major_formatter(formatter)

#Numero de canal
for i in range(0,GENERIC_MEDEL2b.Nchann):
    fc = GENERIC_MEDEL2b.freqsAnalisis[i][0] + GENERIC_MEDEL2b.freqsAnalisis[i][1]
    fc /= 2
    plt.text(fc*0.95, 0.9, f'{i+1}')


plt.show()

Coclear

In [ ]:
fig, ax = plt.subplots(figsize = (12, 7), layout='constrained')
IMPLANT_SR = 16000
Cochlear_Axel_Izq.setearParametros()


s = [1] + [0] * 4095 #delta
outs = filter_bank_analisis(s, Cochlear_Axel_Izq, 6)

for out in outs:
    x = out
    sr = 17000
    fs = np.linspace(0,sr/2, len(x)//2)
    #F = np.abs(fft(x)[:len(x)//2]) / len(x)*2
    F = np.abs(fft(x)[:len(x)//2])
    ax.plot(fs, F)


ax.set_xscale('log')
ax.set_xlim(40)
ax.set_xlabel('Frecuencia $f$ (Hz)', fontsize = 20, font = 'Times New Roman')
ax.set_ylabel('$| H(f) |$', fontsize = 20, font = 'Times New Roman')

ax.set_xticks([50, 100, 200, 500, 1000, 2000, 5000, 8000])
ax.tick_params(axis='both', which='major', labelsize=13)

ax.text
#ax.hlines(1/np.sqrt(2), 0, 8000)  #puntos de interseccion

from matplotlib.ticker import ScalarFormatter
#Hago que los numeros del eje x sean mas lindos
for axis in [ax.xaxis, ax.yaxis]:
    formatter = ScalarFormatter()
    formatter.set_scientific(False)
    axis.set_major_formatter(formatter)

for i in range(0,Cochlear_Axel_Izq.Nchann, 3):
#for i in [0,4,9,14,19]:
    fc = Cochlear_Axel_Izq.freqsAnalisis[i][0] + Cochlear_Axel_Izq.freqsAnalisis[i][1]
    fc /= 2
    plt.text(fc*1.02, 0.9, f'{i+1}')

plt.show()

## Banco de filtros + mismatch de electrodos

In [ ]:
LASTCHANNELHP = True
## Todo esto igual que lo anterior --------------------------------------------------------------------
fig, ax = plt.subplots(figsize = (12, 7), layout='constrained')
IMPLANT_SR = 17000
elec_array = GENERIC_MEDEL3
elec_array.setearParametros()


s = [1] + [0] * 4095 #delta
outs = filter_bank_analisis(s, elec_array, 6)

for out in outs:
    x = out
    sr = 17000
    fs = np.linspace(0,sr/2, len(x)//2)
    #F = np.abs(fft(x)[:len(x)//2]) / len(x)*2
    F = np.abs(fft(x)[:len(x)//2])
    ax.plot(fs, F)


ax.set_xscale('log')
ax.set_xlim(40)
ax.set_xlabel('Frecuencia $f$ (Hz)', fontsize = 20, font = 'Times New Roman')
ax.set_ylabel('$| H(f) |$', fontsize = 20, font = 'Times New Roman')

ax.set_xticks([50, 100, 200, 500, 1000, 2000, 5000, 8500, 12000])
ax.tick_params(axis='both', which='major', labelsize=13)

ax.text
#ax.hlines(1/np.sqrt(2), 0, 8000)  #puntos de interseccion

from matplotlib.ticker import ScalarFormatter
#Hago que los numeros del eje x sean mas lindos
for axis in [ax.xaxis, ax.yaxis]:
    formatter = ScalarFormatter()
    formatter.set_scientific(False)
    axis.set_major_formatter(formatter)

#Numero de canal
for i in range(0,elec_array.Nchann -1):
    fc = elec_array.freqsAnalisis[i]
    plt.text(fc, 0.9, f'{i+1}', fontsize = 12)
plt.text(elec_array.freqsAnalisis[-1]*1.1, 0.9, f'{12}')

## Agrego ahora frecuencias estimadadas de los electrodos   --------------------------------------------------------------------
ax.vlines(elec_array.freqsElectrodos,1.1, 1, linestyles= '--', color = 'r')
for i in range(0,elec_array.Nchann):
    fe = elec_array.freqsElectrodos[i]
    plt.text(fe*0.95, 1.12, "$e_{{{}}}$".format(str(i+1)), color = 'r', fontsize= 15)


Rehago un grafico del informe

## Descomposicion en canales 1

In [ ]:
#Entrada de señal y banco de filtros
GENERIC_MEDEL2b.setearParametros()
IMPLANT_SR = 17000
s, _ = librosa.load('cello B3.wav', sr = IMPLANT_SR)
s/= np.max(np.abs(s)) #Normalizo
outs = filter_bank_CIS(s, GENERIC_MEDEL2b)

#Creo grafico
fig, axs = plt.subplots(9,2, figsize=(10,12))
axs[0][0].plot(s, label = 'Señal de entrada')
axs[0][0].legend(framealpha=1, shadow=True, loc = 'lower left',  prop={'size': 10})
axs[0][0].get_xaxis().set_visible(False)

#Primer grafico
ts = [i/IMPLANT_SR for i in range(len(s))]
F = np.abs(fft(s)/len(s))
freqsfft = [IMPLANT_SR/len(s)*i for i in range(len(s)//2) ]



#Segundo Grafico
axs[0][1].plot(freqsfft, F[:len(s)//2], color = 'darkcyan', label = 'Señal de entrada')
axs[0][1].grid(alpha = 0.5)
axs[0][1].legend(framealpha=1, shadow=True, loc = 'upper right')
axs[0][1].get_xaxis().set_visible(False)
axs[0][1].set_xlim(-120, 3500)

#Canales del banco
for i in range(1,9):
  si = outs[i]
  Fi = np.abs(fft(si)/len(s))

  #Graficos
  axs[i][0].plot(ts, si, color = 'orange', label = 'Canal '+ str(i) + ' ( ' + str(GENERIC_MEDEL2b.freqsAnalisis[i][0]) + 'Hz - ' +str(GENERIC_MEDEL2b.freqsAnalisis[i][1])+ ' Hz)')
  axs[i][1].plot(freqsfft, Fi[:len(s)//2], color = 'fuchsia', label = 'Canal ' + str(i))

  axs[i][1].plot(freqsfft, F[:len(s)//2], color = 'darkcyan', label = 'Señal de entrada', alpha = 0.2)

  axs[i][0].set_ylim(-1,1)
  axs[i][1].set_ylim(-0.01,0.06)
  axs[i][1].set_xlim(-120, 3500)

  axs[i][0].get_xaxis().set_visible(False)
  axs[i][1].get_xaxis().set_visible(False)
  axs[i][0].legend(framealpha=1, shadow=True, loc = 'lower left')
  axs[i][1].legend(framealpha=1, shadow=True, loc = 'upper right')
  axs[i][1].grid(alpha = 0.5)


  #Escucho el audio
  print(f'Canal {i}')
  display(Audio(si, rate = IMPLANT_SR, normalize= False))
  #axs[i][1].plot(si, label = 'Banda ' + str(i+1), alpha = 0.85)



#Nombres de los ejes (abajo de todo)
axs[8][0].set_xlabel('Tiempo (s)', fontsize = 13, font = 'Times New Roman')
axs[8][0].get_xaxis().set_visible(True)
axs[8][1].set_xlabel('Freq (Hz) [DFT]', fontsize = 13, font = 'Times New Roman')
axs[8][1].get_xaxis().set_visible(True)


plt.show()

## Descomposicion en canales 2 espectrograma

In [ ]:
f, t, Sxx = signal.spectrogram(s, IMPLANT_SR,  nperseg=512, noverlap=128)


Sxx_dB = 10 * np.log10(Sxx)
Sxx_dB = np.clip(Sxx_dB, -50, 0)  # Limitar valores entre -100 dB y 0 dB

plt.pcolormesh(t, f, Sxx_dB, cmap = 'magma')
plt.yscale('log')
plt.colorbar(label='Densidad espectral de potencia (dB)')
plt.ylim(100, IMPLANT_SR / 2)  # Establecer límites del eje de frecuencia



Rehago pero con STFT - ESTO ESTA CASI CASI BIEN - el problema es que la escala se resetea en cada grafico. Esto es algo de librosa y es casi

In [ ]:
def draw_stft(x, ax):
  n_fft = 512
  win_length = n_fft
  hop_length = n_fft//4
  top_db = 35

  #matplotlib.colorizer.vmin = -40
  #matplotlib.colorizer.vmax = 0

  #Espectro
  x_stft = np.abs(librosa.stft(x, n_fft=n_fft, win_length = win_length, hop_length = hop_length))
  x_stft_db = librosa.amplitude_to_db(x_stft, top_db = top_db) # ref=np.max normaliza, no quiero
  n_frames = len(x_stft_db[0])

  global vmin
  vmin = min(np.min(x_stft_db),vmin)

  global vmax
  vmax = max(np.max(x_stft_db), vmax)
  print(vmin, vmax)

  sampling_rate = IMPLANT_SR
  cmap = 'magma'
  img = librosa.display.specshow(data=x_stft_db, y_axis='log', x_axis = 'time',ax=ax,
                            sr=sampling_rate, cmap= cmap,
                            n_fft=n_fft, win_length = win_length, hop_length = hop_length, vmin = -0, vmax = 37)
  ax.set_ylim(100,min(14000, sampling_rate/2))

  #ax.pcolormesh(t, f, 10*np.log10(S), shading='gouraud',
  #                vmin=-40, vmax=0)

  return img



#Entrada de señal y banco de filtros
GENERIC_MEDEL2b.setearParametros()
IMPLANT_SR = 17000
s, _ = librosa.load('cornoi.wav', sr = IMPLANT_SR)
s = s[8000:32000]
s/= np.max(np.abs(s)) #Normalizo
outs = filter_bank_CIS(s, GENERIC_MEDEL2b)

#Creo grafico
fig, axs = plt.subplots(3,3, figsize=(10,12),  dpi = 175, sharex = True, sharey = True, layout = 'constrained')
axs = axs.flatten()


#Señal de entrada
draw_stft(s, axs[0])
axs[0].get_xaxis().set_visible(False)
axs[0].set_ylabel("Frec. (Hz)", size = 12)
axs[0].set_title('Señal de entrada')

#Canales del banco
for i in range(1,9):

  x = outs[i]
  x = np.array(x)

  im = draw_stft(x, axs[i] )


  if( i% 3 == 0) :
    axs[i].set_ylabel("Frec. (Hz)", size = 12)
  #for tick in axs[i].yaxis.get_major_ticks():
   #   tick.label.set_fontsize(16)

  label = 'Canal '+ str(i) + '\n (' + str(GENERIC_MEDEL2b.freqsAnalisis[i][0]) + 'Hz - ' +str(GENERIC_MEDEL2b.freqsAnalisis[i][1])+ ' Hz)'
  #Graficos


  #axs[i].set_title(f'Canal {i}' , fontdict = {'size'   : 12} ) #'weight' : 'bold'
  #axs[i].legend(framealpha=1, shadow=True, loc = 'upper right', labels = [label])
  axs[i].set_title(label, fontsize = 11)



  axs[i].grid(alpha = 0.5, axis = 'y')
  if( i<6):
    axs[i].get_xaxis().set_visible(False)
  else:
    axs[i].set_xlabel('Tiempo (s)', fontsize = 13)
    axs[i].set_xticks([0,0.7,1.4])




  #Escucho el audio
  print(f'Canal {i}')
  #display(Audio(x, rate = IMPLANT_SR, normalize= False))

fig.colorbar(im, ax=axs,  format='%+2.0f dB')

#for tick in axs[8].xaxis.get_major_ticks():
#    tick.label.set_fontsize(14)
plt.show()




In [ ]:
def draw_stft_alt(x, ax):
  f, t, Sxx = signal.spectrogram(x, fs=IMPLANT_SR, window='hann', nperseg=8192,
                                scaling='spectrum')
  mx = np.abs(10 * np.log10(Sxx)).max()
  ax.pcolormesh(t, f / 1000, 10 * np.log10(Sxx), shading='gouraud',
                  vmin=-mx, vmax=0)

  return

## Y acá si hecho bien

HORAS, HORAS, COMO 4 HS O MASS!! Para hacer este grafico de M

In [ ]:
vmin, vmax = float('inf'), float('-inf')

In [ ]:
def draw_stft(x, ax):
  n_fft = 512
  win_length = n_fft
  hop_length = n_fft//4
  top_db = 35

  #matplotlib.colorizer.vmin = -40
  #matplotlib.colorizer.vmax = 0

  #Espectro
  x_stft = np.abs(librosa.stft(x, n_fft=n_fft, win_length = win_length, hop_length = hop_length))
  x_stft_db = librosa.amplitude_to_db(x_stft, top_db = top_db) # ref=np.max normaliza, no quiero
  x_stft_db -= 38 #normalizo a 0db el maximo. Este 38 ya lo averigue al correr antes, es vmax
  n_frames = len(x_stft_db[0])

  global vmin
  vmin = min(np.min(x_stft_db),vmin)

  global vmax
  vmax = max(np.max(x_stft_db), vmax)
  print(vmin, vmax)

  sampling_rate = IMPLANT_SR
  cmap = 'magma'
  img = librosa.display.specshow(data=x_stft_db, y_axis='log', x_axis = 'time',ax=ax,
                            sr=sampling_rate, cmap= cmap,
                            n_fft=n_fft, win_length = win_length, hop_length = hop_length, vmin = -38, vmax = 0)
  ax.set_ylim(100,min(14000, sampling_rate/2))

  #ax.pcolormesh(t, f, 10*np.log10(S), shading='gouraud',
  #                vmin=-40, vmax=0)

  return img



#Entrada de señal y banco de filtros
GENERIC_MEDEL2b.setearParametros()
IMPLANT_SR = 17000
s, _ = librosa.load('cornoi.wav', sr = IMPLANT_SR)
s = s[8000:32000]
s/= np.max(np.abs(s)) #Normalizo
outs = filter_bank_CIS(s, GENERIC_MEDEL2b)

#Creo grafico
fig, axs = plt.subplots(3,3, figsize=(10,12),  dpi = 175, sharex = True, sharey = True, layout = 'constrained')
axs = axs.flatten()


#Señal de entrada
draw_stft(s, axs[0])
axs[0].get_xaxis().set_visible(False)
axs[0].set_ylabel("Frec. (Hz)", size = 12)
axs[0].set_title('Señal de entrada')

#Canales del banco
for i in range(8):

  ax = axs[i+1]

  x = outs[i]
  x = np.array(x)

  im = draw_stft(x, ax )


  if( (i+1)% 3 == 0) :
    ax.set_ylabel("Frec. (Hz)", size = 12)
  else:
    ax.set_ylabel('')
  #for tick in axs[i].yaxis.get_major_ticks():
   #   tick.label.set_fontsize(16)

  label = 'Canal '+ str(i+1) + '\n (' + str(GENERIC_MEDEL2b.freqsAnalisis[i][0]) + 'Hz - ' +str(GENERIC_MEDEL2b.freqsAnalisis[i][1])+ ' Hz)'
  #Graficos


  #axs[i].set_title(f'Canal {i}' , fontdict = {'size'   : 12} ) #'weight' : 'bold'
  #axs[i].legend(framealpha=1, shadow=True, loc = 'upper right', labels = [label])
  ax.set_title(label, fontsize = 11)



  ax.grid(alpha = 0.5, axis = 'y')
  if( i+1<6):
    ax.get_xaxis().set_visible(False)
  else:
    ax.set_xlabel('Tiempo (s)', fontsize = 13)
    ax.set_xticks([0,0.7,1.4])




  #Escucho el audio
  print(f'Canal {i+1}')
  display(Audio(x, rate = IMPLANT_SR, normalize= False))

fig.colorbar(im, ax=axs,  format='%+2.0f dB', aspect = 40)

#for tick in axs[8].xaxis.get_major_ticks():
#    tick.label.set_fontsize(14)
plt.show()


In [ ]:
plt.plot(outs[6][:1000])

## Envolventes Medel

In [ ]:
def analytic(x, circular=True):

  n = len(x) #Voy a apendear n zeros, así que duplico el tamaño
  if(not circular): n *= 2
  Z = np.zeros(n, dtype='complex')

  #Aplico una ventana para que la función periodizada sea continua - sino la envolvente exlplota si se extrae sobre muestras muy largas
  #x = np.array(x)* Tukey_window(len(x), 0.1)

  if(circular): X  = fft(x)
  else: X = fft( list(x) + len(x)*[0]) #acá appendeo los n 0's

  for i in range(1,n//2): Z[i] = 2.*X[i]
  Z[n//2] = X[n//2]
  z = ifft(Z)
  return z

def HT(x, circular = True):
  return np.imag(analytic(x, circular))

def lowpass_zerophase(x):
      b,a = signal.butter(2, 0.01, 'lp', output='ba')
      return signal.filtfilt(b,a, x)

FILTER_ENVELOPE = False

In [ ]:
def graficarTodo(x, sr, ti = 0, tf = None):
  a = analytic(x)
  env = np.abs(a)
  if(FILTER_ENVELOPE): env = lowpass_zerophase(env)

  t = np.linspace(0, len(x)/sr, len(x))

  fig, axs = plt.subplots(1, figsize = (15,4))
  #axs.set_title('Señal x(t) ;  Envolvente', fontsize = 13)
  axs.plot(t, x, color ='b', label= 'x(t)')
  axs.set_xlim([ti, tf])

  axs.plot(t, env, 'r--', label='Envolvente', linewidth = 2, alpha = 0.6) # color='r'
  axs.set_xlabel('Tiempo (s)', fontsize = 14)
  axs.set_ylabel('Amplitud', fontsize = 14)
  #axs.plot(-1*env[:rango], 'r--') # color='r'
  plt.legend(prop={'size': 12})

  plt.show()

In [ ]:
x = outs[3]


display(Audio(x, rate = IMPLANT_SR, normalize= False))

graficarTodo(x, IMPLANT_SR,  ti = 0, tf = 1.2)
graficarTodo(x, IMPLANT_SR,  ti = 0.4, tf = 0.5)

In [ ]:
x = outs[4]


display(Audio(x, rate = IMPLANT_SR, normalize= False))

graficarTodo(x, IMPLANT_SR,  ti = 0, tf = 1.2)
graficarTodo(x, IMPLANT_SR,  ti = 0.4, tf = 0.5)

## Envolventes Cochlear

Tener en cuenta que este método, para sinusoides, necesita un factor de escala de pi/2 para compensar

In [ ]:
x = puretone(500, 2, IMPLANT_SR)
env = envelope_Cochlear(x, 50, fullwave = True)
plt.plot(x)
plt.plot(env)
plt.plot(env*np.pi/2, color = 'r')

In [ ]:
IMPLANT_SR = 16000
ENVELOPE_ZERO_PHASE = False

x=outs[2]
display(Audio(x, rate =IMPLANT_SR, normalize=False))

env_calc = envelope_Cochlear(x, fc = 200, fullwave = True, gain_compensation= False)

t = np.linspace(0, len(x)/IMPLANT_SR, len(x))

plt.plot(t,x, label ='Señal')
#plt.plot(env_calc)
plt.plot(t,env_calc*np.pi/2, label = 'Envolvente', color = 'r', alpha = 0.8)

plt.xlabel('Tiempo (s)', fontsize = 13)
plt.ylabel('Amplitud', fontsize = 13)

plt.legend(prop = {'size': 11})

A partir de abajo ya uso la version corregida de env, que agrega este factorcito cuando se hace fullwave rect

In [ ]:
elec_array = GENERIC_COCHLEAR1

#Entrada de señal y banco de filtros
elec_array.setearParametros()
IMPLANT_SR = 16000

s, _ = librosa.load('cornoi.wav', sr = IMPLANT_SR)
s = s[8000:30000]
s/= np.max(np.abs(s)) #Normalizo
outs = filter_bank_CIS(s, elec_array)

#Creo grafico
fig, axs = plt.subplots(3,3, figsize=(10,12),  dpi = 175, sharex = True, sharey = True, layout = 'constrained')
axs = axs.flatten()


#Señal de entrada
draw_stft(s, axs[0])
axs[0].get_xaxis().set_visible(False)
axs[0].set_ylabel("Frec. (Hz)", size = 12)
axs[0].set_title('Señal de entrada')

#Canales del banco
for i in range(8):

  ax = axs[i+1]

  x = outs[i*2]
  x = np.array(x)

  im = draw_stft(x, ax )


  if( (i+1)% 3 == 0) :
    ax.set_ylabel("Frec. (Hz)", size = 12)
  else:
    ax.set_ylabel('')
  #for tick in axs[i].yaxis.get_major_ticks():
   #   tick.label.set_fontsize(16)

  label = 'Canal '+ str(i*2+1) + '\n (' + str(elec_array.freqsAnalisis[2*i][0]) + 'Hz - ' +str(elec_array.freqsAnalisis[2*i][1])+ ' Hz)'
  #Graficos


  #axs[i].set_title(f'Canal {i}' , fontdict = {'size'   : 12} ) #'weight' : 'bold'
  #axs[i].legend(framealpha=1, shadow=True, loc = 'upper right', labels = [label])
  ax.set_title(label, fontsize = 11)



  ax.grid(alpha = 0.5, axis = 'y')
  if( i+1<6):
    ax.get_xaxis().set_visible(False)
  else:
    ax.set_xlabel('Tiempo (s)', fontsize = 13)
    ax.set_xticks([0,0.7,1.4])




  #Escucho el audio
  print(f'Canal {2*i+1}')
  display(Audio(x, rate = IMPLANT_SR, normalize= False))

fig.colorbar(im, ax=axs,  format='%+2.0f dB', aspect = 40)

#for tick in axs[8].xaxis.get_major_ticks():
#    tick.label.set_fontsize(14)
plt.show()


In [ ]:
ENVELOPE_ZERO_PHASE = False

x=outs[6]
display(Audio(x, rate =IMPLANT_SR, normalize=False))

env_calc = envelope_Cochlear(x, fc = 200, fullwave = True, gain_compensation= False)

t = np.linspace(0, len(x)/IMPLANT_SR, len(x))
fig, ax = plt.subplots(1, figsize = (5,3))

plt.plot(t,x, label ='Salida Canal 7')
#plt.plot(env_calc)
plt.plot(t,env_calc*np.pi/2, label = 'Envolvente', color = 'r', alpha = 0.8)

plt.xlabel('Tiempo (s)', fontsize = 13)
plt.ylabel('Amplitud', fontsize = 13)

plt.legend(prop = {'size': 11}, loc = 'lower left')

## N of M

In [ ]:
def nofm(envs, elec_array, power, m):

    #Recibe envolventes de cada canal normal y deveulve las mismas envolventes pero haciendo 0 todo lo que ACE no selecciona

    #power = 1 me dice usá promedio
    #power = 2 me dice usa RMS

    nSamps = len(envs[0])
    outs = np.zeros([elec_array.Nchann, nSamps])

    WIN_SIZE = round(WIN_LEN_ACE * IMPLANT_SR/1000) #en sampless, Win len en ms

    envACE = Tukey(WIN_SIZE, 0.25) #para encender los nuevos
    envOn = envACE.copy()
    envOn[WIN_SIZE//2: WIN_SIZE] = np.ones(WIN_SIZE//2)

    envOff = envACE.copy()
    envOff[0:WIN_SIZE//2] = np.ones(WIN_SIZE//2)


    encendidosAnterior = [] #canales seleccionados en el ciclo anterior. No se les aplica ventana.
    i = 0
    while(i+WIN_SIZE<nSamps):

      #Extraigo las envolventes de mayor amplitud
      amplitudes_promedio = [ sum(np.array(envs[c][i: i+WIN_SIZE]**power))**(1/power)  for c in range(elec_array.Nchann)] #Tomo la amplitud promedio en el ciclo de estimulación de cada envolvente
      indices_max = np.argpartition(amplitudes_promedio, -m)[-m:] #Indices de las M envolventes de mayor amplitud

      #Estimulo bandas maximas
      for max_channel in indices_max:

        outs[max_channel][i: i+WIN_SIZE] = envs[max_channel][i: i+WIN_SIZE]

      i+=WIN_SIZE

    return outs


In [ ]:
elec_array = GENERIC_COCHLEAR1

WIN_LEN_ACE = 4 #ms
IMPLANT_SR = 16000
MAXIMA = 5

elec_array.setearParametros()

#Señal a usar
s, _ = librosa.load('cornoi.wav', sr = IMPLANT_SR)

ni = 8000 #sample inicial
nf = 12000 #sample final

s = s[ni:nf]
s/= np.max(np.abs(s)) #Normalizo
outs = filter_bank_CIS(s, elec_array) #Obtengo la salida de los filtros
t = [i/IMPLANT_SR for i in range(ni, nf)]

envs =[]
for out in outs:
  envs.append(envelope_Cochlear(out, fc = 200, fullwave = True, gain_compensation= False))


#Calculo envolventes seleccionadaas con ACE
envs_out = nofm(envs,elec_array, 1, MAXIMA)



#Grafico original
fig, axs = plt.subplots(9,1, figsize = (12,9))
axs[0].plot(t, s, label = 'Señal de entrada')
axs[0].legend(framealpha=0.8, shadow=True, loc = 'lower right',  prop={'size': 9})
axs[0].set_xticks([]) #saco ticks

#Grafico envolvente de los canales
for i in range(8):

  ax = axs[i+1]

  ax.plot(t, envs_out[i], color = 'black', label = 'Canal' + str(i+1))
  ax.set_ylim(-0.08, 0.5)

  ax.legend(framealpha=0.8, shadow=True, loc = 'lower right',  prop={'size': 9})

  ax.hlines(0, ni/IMPLANT_SR, nf/IMPLANT_SR, color = 'grey', alpha = 0.4, linestyle='--')

  ax.set_xticks([]) #saco ticks
  #axs[i].grid(alpha = 0.5)


from matplotlib.ticker import AutoLocator
axs[-1].xaxis.set_major_locator(AutoLocator()) #Pongo ticks por defectos de tiempo abajo de todo

plt.xlabel('Tiempo (s)', fontsize = 14)
axs[4].set_ylabel('Amplitud', fontsize = 14)
plt.show()
display(Audio(s, rate = IMPLANT_SR))


Canales impares

In [ ]:
elec_array = GENERIC_COCHLEAR1

WIN_LEN_ACE = 4 #ms
IMPLANT_SR = 16000
MAXIMA = 5

elec_array.setearParametros()

#Señal a usar
s, _ = librosa.load('cornoi.wav', sr = IMPLANT_SR)

ni = 8000 #sample inicial
nf = 12000 #sample final

s = s[ni:nf]
s/= np.max(np.abs(s)) #Normalizo
outs = filter_bank_CIS(s, elec_array) #Obtengo la salida de los filtros
t = [i/IMPLANT_SR for i in range(ni, nf)]

envs =[]
for out in outs:
  envs.append(envelope_Cochlear(out, fc = 200, fullwave = True, gain_compensation= False))


#Calculo envolventes seleccionadaas con ACE
envs_out = nofm(envs,elec_array, 1, MAXIMA)



#Grafico original
fig, axs = plt.subplots(9,1, figsize = (12,9))
axs[0].plot(t, s, label = 'Señal de entrada')
axs[0].legend(framealpha=0.8, shadow=True, loc = 'lower right',  prop={'size': 9})
axs[0].set_xticks([]) #saco ticks

#Grafico envolvente de los canales
for i in range(8):

  ax = axs[i+1]

  ax.plot(t, envs_out[2*i], color = 'black', label = 'Canal' + str(2*i+1))
  ax.set_ylim(-0.08, 0.5)

  ax.legend(framealpha=0.8, shadow=True, loc = 'lower right',  prop={'size': 9})

  ax.hlines(0, ni/IMPLANT_SR, nf/IMPLANT_SR, color = 'grey', alpha = 0.4, linestyle='--')

  ax.set_xticks([]) #saco ticks
  #axs[i].grid(alpha = 0.5)


from matplotlib.ticker import AutoLocator
axs[-1].xaxis.set_major_locator(AutoLocator()) #Pongo ticks por defectos de tiempo abajo de todo

plt.xlabel('Tiempo (s)', fontsize = 14)
axs[4].set_ylabel('Amplitud', fontsize = 14)
plt.show()
display(Audio(s, rate = IMPLANT_SR))


Todos los canales

In [ ]:
elec_array = GENERIC_COCHLEAR1

WIN_LEN_ACE = 4 #ms
IMPLANT_SR = 16000
MAXIMA = 5

elec_array.setearParametros()

#Señal a usar
s, _ = librosa.load('cornoi.wav', sr = IMPLANT_SR)

ni = 8000 #sample inicial
nf = 12000 #sample final

s = s[ni:nf]
s/= np.max(np.abs(s)) #Normalizo
outs = filter_bank_CIS(s, elec_array) #Obtengo la salida de los filtros
t = [i/IMPLANT_SR for i in range(ni, nf)]

envs =[]
for out in outs:
  envs.append(envelope_Cochlear(out, fc = 200, fullwave = True, gain_compensation= False))


#Calculo envolventes seleccionadaas con ACE
envs_out = nofm(envs,elec_array, 1, MAXIMA)



#Grafico original
fig, axs = plt.subplots(23,1, figsize = (12,9))
axs[0].plot(t, s, label = 'Señal de entrada')
axs[0].legend(framealpha=0.8, shadow=True, loc = 'lower right',  prop={'size': 9})
axs[0].set_xticks([]) #saco ticks

#Grafico envolvente de los canales
for i in range(22):

  ax = axs[i+1]

  ax.plot(t, envs_out[i], color = 'black', label = 'Canal' + str(i+1))
  ax.set_ylim(-0.08, 0.5)

  ax.legend(framealpha=0.8, shadow=True, loc = 'lower right',  prop={'size': 9})

  ax.hlines(0, ni/IMPLANT_SR, nf/IMPLANT_SR, color = 'grey', alpha = 0.4, linestyle='--')

  ax.set_xticks([]) #saco ticks
  #axs[i].grid(alpha = 0.5)


from matplotlib.ticker import AutoLocator
axs[-1].xaxis.set_major_locator(AutoLocator()) #Pongo ticks por defectos de tiempo abajo de todo

plt.xlabel('Tiempo (s)', fontsize = 14)
axs[4].set_ylabel('Amplitud', fontsize = 14)
plt.show()
display(Audio(s, rate = IMPLANT_SR))


Medel

In [ ]:
elec_array = GENERIC_MEDEL3

WIN_LEN_ACE = 4 #ms
IMPLANT_SR = 17000
MAXIMA = 6

elec_array.setearParametros()

#Señal a usar
s, _ = librosa.load('cornoi.wav', sr = IMPLANT_SR)

ni = 8000 #sample inicial
nf = 12000 #sample final

s = s[ni:nf]
s/= np.max(np.abs(s)) #Normalizo
outs = filter_bank_CIS(s, elec_array) #Obtengo la salida de los filtros
t = [i/IMPLANT_SR for i in range(ni, nf)]

envs =[]
for out in outs:
  envs.append(envelope_Cochlear(out, fc = 200, fullwave = True, gain_compensation= False))


#Calculo envolventes seleccionadaas con ACE
envs_out = nofm(envs,elec_array, 1, MAXIMA)



#Grafico original
fig, axs = plt.subplots(9,1, figsize = (12,9))
axs[0].plot(t, s, label = 'Señal de entrada')
axs[0].legend(framealpha=0.8, shadow=True, loc = 'lower right',  prop={'size': 9})
axs[0].set_xticks([]) #saco ticks

#Grafico envolvente de los canales
for i in range(1,9):

  axs[i].plot(t, envs_out[i-1], color = 'black', label = 'Canal' + str(i))
  axs[i].set_ylim(-0.08, 0.5)

  axs[i].legend(framealpha=0.8, shadow=True, loc = 'lower right',  prop={'size': 9})

  axs[i].hlines(0, ni/IMPLANT_SR, nf/IMPLANT_SR, color = 'grey', alpha = 0.4, linestyle='--')

  axs[i].set_xticks([]) #saco ticks
  #axs[i].grid(alpha = 0.5)


from matplotlib.ticker import AutoLocator
axs[-1].xaxis.set_major_locator(AutoLocator()) #Pongo ticks por defectos de tiempo abajo de todo

plt.xlabel('Tiempo (s)', fontsize = 14)
axs[4].set_ylabel('Amplitud', fontsize = 14)
plt.show()
display(Audio(s, rate = IMPLANT_SR))

## Frequency maps en OC y SG

In [ ]:
#pip install import-ipynb
import import_ipynb
#import CIS_6
from CIS_6 import *

Ajuste cubico de procentaje de distancia entre OC y SG (Divya Sridhar, Olga Stakhovskaya 2005)

Ajuste de procentaje de distancia entre OC y SG v2 (Olga Stakhovskaya 2006)

In [ ]:
OC_p = np.linspace(1,100,100)

MW_p = OC_to_MW_p(OC_p)
SG_p_old= OC_to_SG_p(OC_p, metodo = 1)
SG_p    = OC_to_SG_p(OC_p, metodo = 2)

plt.plot(OC_p, SG_p, label = 'SG (2006)', alpha = 0.7)
plt.plot(OC_p, SG_p_old, label = 'SG (2005)', alpha = 0.7)
plt.plot(OC_p, MW_p, label = 'MW', alpha = 0.7)
plt.xlabel('Proporcion de OC distance (%)')
plt.ylabel('Proporcion de SG distance (%)')
plt.legend()
plt.grid()
plt.show()


In [ ]:
angs = np.linspace(0,360*2.5,100)

OC_p = ang_to_OC_p(angs)
SG_p = ang_to_SG_p(angs)

plt.plot(angs, OC_p, label = 'OC', alpha = 0.7)
plt.plot(angs, SG_p, label = 'SG', alpha = 0.7)
plt.xlabel('Ángulo de inserción (°)')
plt.ylabel('Proporcion recorrida (%)')
plt.legend()
plt.grid()
plt.show()


Inversa de esto: Dos métodos (metodo numerico y metodo de modelo de angulos)

In [ ]:
pip install pynverse

In [ ]:
p_SG = np.linspace(1, 100, 100)


#Metodo 1: Inversa numerica
p_OC1 = SG_p_to_OC_p(p_SG, metodo = 1)

#Metodo 2: Composicion de funciones y el modelo del angulo
p_OC2 = SG_p_to_OC_p(p_SG, metodo = 2)

plt.plot(p_SG, p_OC1, label = 'Metodo 1 (inversa numérica)')
plt.plot(p_SG, p_OC2, label = 'Metodo 2 (inversa con modelo)')


plt.xlabel('Proporcion SG (%)')
plt.ylabel('Proporcion OC (%)')
plt.legend()
plt.grid()
plt.show()

In [ ]:
p = np.linspace(0.001, 1, 100)


#Metodo 1: Inversa numerica
freqs_SG1 = Greenwood_Perimodiolar_SG_p(p, metodo = 1)

#Metodo 2: Composicion de funciones y el modelo del angulo
freqs_SG2 = Greenwood_Perimodiolar_SG_p(p, metodo = 2)

#Organo de Corti
freqs_OC = Greenwood_p(p)


plt.plot(p*100, freqs_SG1, label = 'SG (inversa numérica)')
plt.plot(p*100, freqs_SG2, label = 'SG (inversa con modelo)')
plt.plot(p*100, freqs_OC, label = 'OC')



plt.xlabel('Proporcion recorrida desde la base (%)')
plt.ylabel('Freq (Hz)')
plt.legend()


plt.grid()
plt.show()

Lo mismo pero en logaritmico y para usar en la tesis

In [ ]:
p = np.linspace(0.001, 1, 100)


#Metodo 1: Inversa numerica
freqs_SG1 = Greenwood_Perimodiolar_SG_p(p, metodo = 1)

#Metodo 2: Composicion de funciones y el modelo del angulo
freqs_SG2 = Greenwood_Perimodiolar_SG_p(p, metodo = 2)

#Organo de Corti
freqs_OC = Greenwood_p(p)

#pared modilar
freqs_MW = Greenwood_Perimodiolar_MW_p(p)


plt.plot(p*100, freqs_SG1, label = 'Ganglio espiral (Método numérico)', linestyle = ':')
plt.plot(p*100, freqs_SG2, label = 'Ganglio espiral (Modelo angular)', linestyle = '-.')
plt.plot(p*100, freqs_OC, label = 'Órgano de Corti (Greenwood)', linestyle = '--')
plt.plot(p*100, freqs_MW, label = 'Pared Modiolar (Método numérico)', color = 'r', linestyle = '-')



plt.xlabel('Proporcion recorrida desde la base (%)', fontsize = 12)
plt.ylabel('Frec. (Hz)', fontsize = 12)
plt.legend()
plt.yscale('log')
plt.yticks([20,100,500,1000,2000,5000,10000, 15000])

ax = plt.gca()
ax.tick_params(axis='both', which='major', labelsize=11)

#ax.set_ylim(40)

from matplotlib.ticker import ScalarFormatter
#Hago que los numeros del eje x sean mas lindos
for axis in [ax.xaxis, ax.yaxis]:
    formatter = ScalarFormatter()
    formatter.set_scientific(False)
    axis.set_major_formatter(formatter)

plt.grid()
plt.show()

In [ ]:

p = np.linspace(0.001, 1, 100)


#Metodo 1: Inversa numerica
freqs_SG1 = Greenwood_Perimodiolar_SG_p(p, metodo = 1)

#Metodo 2: Composicion de funciones y el modelo del angulo
freqs_SG2 = Greenwood_Perimodiolar_SG_p(p, metodo = 2)

#Organo de Corti
freqs_OC = Greenwood_p(p)

#pared modilar
freqs_MW = Greenwood_Perimodiolar_MW_p(p)


plt.plot(p*100, freqs_SG1, label = 'Ganglio espiral', linestyle = ':')
plt.plot(p*100, freqs_OC, label = 'Órgano de Corti', linestyle = '--', color = 'g')
plt.plot(p*100, freqs_MW, label = 'Pared Modiolar', color = 'r', linestyle = '-')



plt.xlabel('Proporcion recorrida desde la base (%)', fontsize = 12)
plt.ylabel('Frec. (Hz)', fontsize = 12)
plt.legend()
plt.yscale('log')
plt.yticks([20,100,500,1000,2000,5000,10000, 15000])

ax = plt.gca()
ax.tick_params(axis='both', which='major', labelsize=11)

#ax.set_ylim(40)

from matplotlib.ticker import ScalarFormatter
#Hago que los numeros del eje x sean mas lindos
for axis in [ax.xaxis, ax.yaxis]:
    formatter = ScalarFormatter()
    formatter.set_scientific(False)
    axis.set_major_formatter(formatter)

plt.grid()
plt.show()

## |F(s)| de un pulso muy cortito

In [ ]:
dur = 156* 10**-6 #duracion del pulso en ms
def sinc(x):
    if x==0 : return 1
    return np.sin(pi*x) / pi/x

fs = np.linspace(0,20000, 500)
F =  [sinc(f*dur) for f in fs] #tecnicamente hay que multiplicar por dur pero lo normalizo a 1 acá

plt.plot(fs,20*np.log10(np.abs(F)))
plt.xlabel('Frec. (Hz)')
plt.ylabel('$|F|$ (dB)')


## Frecuencias en un perimodilar vs Lateral

In [ ]:
n = range(22)
plt.errorbar(n, Cochlear_Axel_Izq.freqsElectrodos, xerr = 0, yerr = 200, label = 'Modiolar',  capsize=3, fmt="c--o", ecolor = "black", capthick=1, ms = 3)
plt.errorbar(n, Cochlear_Axel_Der.freqsElectrodos, label = 'Lateral')
plt.xlabel('N° Electrodo')
plt.ylabel('Frec. (Hz)')
plt.grid()
plt.yscale('log')
plt.legend()

In [ ]:
n = range(22)
plt.errorbar(n, Cochlear_Axel_Izq.freqsElectrodos, xerr = 0, yerr = 200, label = 'Modiolar',  capsize=3, fmt="c--o", ecolor = "black", capthick=1, ms = 3)
plt.errorbar(n, Cochlear_Axel_Der.freqsElectrodos, label = 'Lateral')
plt.xlabel('N° Electrodo')
plt.ylabel('Frec. (Hz)')
plt.grid()
plt.yscale('log')
plt.legend()

## Dispersion v2

In [ ]:
import numpy as np
from numpy import pi, sin, cos, abs, log2
from scipy.fft import fft, ifft
from scipy.io import wavfile
from scipy.signal import lfilter
from scipy import signal
import librosa, librosa.display
import matplotlib.pyplot as plt
import os

import import_ipynb
import CIS_6

Dispersion de antes

In [ ]:

d = [1] + [0]*50000 #delta

elec_array = CIS_6.Cochlear_Axel_Der
CIS_6.DISPERTION = 0.5 #parametro de dispersion
CIS_6.EXP_OYENTE = True #lo hago para mi

outs = CIS_6.filter_bank_sintesis(d, elec_array, orden = 1)
fs = np.linspace(0, CIS_6.OUT_SR/2, len(outs[0])//2)

for out in outs:
    F = fft(out)[0:len(out)//2]
    plt.plot(fs, np.abs(F))

plt.xlim(0,12000)
plt.xlabel('Freq')
plt.ylabel('Amp')
plt.title(f'Dispertion = {CIS_6.DISPERTION}')

Dispersion nueva

In [ ]:
import import_ipynb
import CIS_6

d = [1] + [0]*96000 #delta

elec_array = CIS_6.Cochlear_Axel_Der
CIS_6.D = 0.2 #parametro de dispersion
#CIS_6.EXP_OYENTE = True #no hace nada en v2 esto
CIS_6.ASYMMETRY_INDEX = 0 #simetria

outs = CIS_6.filter_bank_sintesis2(d, elec_array)
fs = np.linspace(0, CIS_6.OUT_SR/2, len(outs[0])//2)

for out in outs:
    F = fft(out)[0:len(out)//2]
    plt.plot(fs, np.abs(F))

plt.xlim(0,12000)
plt.xlabel('Freq')
plt.ylabel('Amp')
plt.title(f'D = {CIS_6.D}')



In [ ]:
import import_ipynb
import CIS_6

d = [1/0.6] + [0]*96000 #delta

CIS_6.OUT_SR = 44100

elec_array = CIS_6.GENERIC_MEDEL3
CIS_6.D = 0.3 #parametro de dispersion
#CIS_6.EXP_OYENTE = True #no hace nada en v2 esto
CIS_6.ASYMMETRY_INDEX = 0.9 #simetria

outs_medel = CIS_6.filter_bank_sintesis2(d, elec_array)
fs = np.linspace(0, CIS_6.OUT_SR/2, len(outs_medel[0])//2)

elec_array = CIS_6.Cochlear_Axel_Der
outs_cochlear = CIS_6.filter_bank_sintesis2(d, elec_array)


fig, axs = plt.subplots(1,2 , figsize =(12,5), layout = 'constrained')

for out in outs_medel:
    F = fft(out)[0:len(out)//2]
    axs[0].plot(fs, np.abs(F))

for out in outs_cochlear:
    F = fft(out)[0:len(out)//2]
    axs[1].plot(fs, np.abs(F))

from matplotlib.ticker import ScalarFormatter

for ax in axs:

    ax.set_xlim(40,20000)
    ax.set_xlabel('Frec. (Hz)', fontsize = 12)
    ax.set_xscale('log')

    ax.set_xticks([50,100, 200, 500, 1000, 2000, 5000, 10000, 20000])
    ax.tick_params(axis='both', which='major', labelsize=11)
    ax.get_xaxis().set_major_formatter(ScalarFormatter())

axs[0].set_title(f'Flex28 - D = {CIS_6.D} mm (dx = {CIS_6.GENERIC_MEDEL3.dx}mm )')
axs[1].set_title(f'Nucleus24 - D = {CIS_6.D} mm (dx = {CIS_6.Cochlear_Axel_Der.dx}mm)')

axs[0].set_ylabel('Amp', fontsize = 12)




Esto es raro??? Es asi realmente????

In [ ]:
import import_ipynb
import CIS_6

d = [1/0.6] + [0]*96000 #delta

CIS_6.OUT_SR = 44100

elec_array = CIS_6.GENERIC_MEDEL3
CIS_6.D = 0.3 #parametro de dispersion
#CIS_6.EXP_OYENTE = True #no hace nada en v2 esto
CIS_6.ASYMMETRY_INDEX = 0.95 #simetria

outs_medel = CIS_6.filter_bank_sintesis2(d, elec_array)
fs = np.linspace(0, CIS_6.OUT_SR/2, len(outs_medel[0])//2)

elec_array = CIS_6.Cochlear_Axel_Der
outs_cochlear = CIS_6.filter_bank_sintesis2(d, elec_array)


fig, axs = plt.subplots(1,2 , figsize =(12,5), layout = 'constrained')

for out in outs_medel:
    F = fft(out)[0:len(out)//2]
    axs[0].plot(fs, 20*np.log10(np.abs(F)))

for out in outs_cochlear:
    F = fft(out)[0:len(out)//2]
    axs[1].plot(fs, 20*np.log10(np.abs(F)))

from matplotlib.ticker import ScalarFormatter

for ax in axs:

    ax.set_xlim(40,20000)
    ax.set_xlabel('Frec. (Hz)', fontsize = 12)
    ax.set_xscale('log')

    ax.set_xticks([50,100, 200, 500, 1000, 2000, 5000, 10000, 20000])
    ax.tick_params(axis='both', which='major', labelsize=11)
    ax.get_xaxis().set_major_formatter(ScalarFormatter())
    ax.set_ylim(-60,6)

axs[0].set_title(f'Flex28 - D = {CIS_6.D} mm (dx = {CIS_6.GENERIC_MEDEL3.dx}mm )')
axs[1].set_title(f'Nucleus24 - D = {CIS_6.D} mm (dx = {CIS_6.Cochlear_Axel_Der.dx}mm)')

axs[0].set_ylabel('Amp (dB)', fontsize = 12)




In [ ]:
from scipy import signal

d = [1]*100000
sos = signal.butter(20, 4285, 'highpass', fs=44100, output='sos')
h = signal.sosfilt(sos, d)
sos = signal.butter(20, 4285, 'lowpass', fs=44100, output='sos')
h = signal.sosfilt(sos, h)

s = CIS_6.puretone(4285, 2, 44100)
CIS_6.plot_fft(h, elec_array, f_from = 0, f_to = 5000, range_dB = 60, sr = 44100, showElecs= True, show= False, logx=False)

## Reasignacion (redo)

In [ ]:
import import_ipynb
from CIS_6 import *
from CIS_6_Best_Match_v4 import *

In [ ]:
#Hago la dft de s en frecuencias especificas
#Bien, pero si la señal es de ruido puede andar mal.... mejor integrar la fft en un rango de frecuencias como hago abajo
def espectrosEspecificos(s, freqs, sr, norm = True):

    dur = len(s)/sr
    Fs = []
    for f in freqs:
        sinsig = puretone(f,dur, sr, phase = 0)
        cossig = puretone(f,dur, sr, phase = 90)

        A = np.dot(sinsig, s)**2 + np.dot(cossig, s)**2
        A = np.sqrt(A)
        Fs.append(A)
    Fs = np.array(Fs)
    if(norm): Fs/=max(Fs)
    return Fs

In [ ]:
from scipy.fft import fft, fftfreq
def integrate_fft(signal, sampling_rate, fcs, bw=20): #copiado para acelerar
    Fs = []
    for fc in fcs:
        fmin = fc-bw/2
        fmax = fc + bw/2
        """Computes the integral (sum) of the FFT magnitude in the frequency range [f_min, f_max]."""
        N = len(signal)
        freq = fftfreq(N, d=1/sampling_rate)  # Frequency bins
        fft_values = fft(signal)
        magnitude = np.abs(fft_values)[:N//2]  # Take positive frequencies only
        freq = freq[:N//2]  # Positive frequency range

        # Select indices within the given frequency range
        mask = (freq >= fmin) & (freq <= fmax)
        Fs.append(np.sum(magnitude[mask]))
    return np.array(Fs)

def integrate_fft_bins(signal, sr, fcs, nbins=5):
    Fs = []
    N = len(signal)
    freq = fftfreq(N, d=1/sr)  # Frequency bins
    dbin = sr/N

    for fc in fcs:
        fmin = fc- nbins*dbin/2
        fmax = fc + nbins*dbin/2
        """Computes the integral (sum) of the FFT magnitude in the frequency range [f_min, f_max]."""
        fft_values = fft(signal)
        magnitude = np.abs(fft_values)[:N//2]  # Take positive frequencies only
        freq = freq[:N//2]  # Positive frequency range

        # Select indices within the given frequency range
        mask = (freq >= fmin) & (freq <= fmax)
        Fs.append(np.sum(magnitude[mask]))
    return np.array(Fs)

def espectrosEspecificos2(s, freqs, sr, norm = True):
    dur = len(s)/sr

    Fs = integrate_fft_bins(s, sr, freqs)

    if(norm): Fs/=max(Fs)
    return Fs


In [ ]:
#Defino la nota que quiero
import pickle
with open('output una nota Medel.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    nota_puntuada_porclase = pickle.load(f)



N = 6
puesto = 0

CIS_6.DEBUG = True

elec_array = GENERIC_MEDEL3
melo_nota = nota_puntuada_porclase[N][puesto]
melo_nota.printInformeScores()

nota = melo_nota.notas[0]
nota.vibDepth=0
nota.dur = 2 #hago que dure mas solo para tener mas puntos en las fft despues
melo_nota.dur = 2

sig = melo_nota.sintetizar(44100)


origSin, origNoise, outOyenteSin, outOyenteNoise , outCISin2, outCINoise2 = elec_array.generarSenialesMelodia(melo_nota, sr = 44100)
# 44100    44100        44100           44100          elecSR      elecSR
#sig == origSin

#genero la señal en los electrodos y desp analizo la energia en cada uno con y sin reasignacion
CIS_6.D = 0.02
out_normal = elec_array.CIS(sig, 44100)
out_reasignada = elec_array.CIS(outCISin2, elec_array.SR)

elec_array.generarTodasSeniales(melo_nota, sr = 44100)

Esta nota de por si sola esta casi totalmente alineada con los electrodos

In [ ]:
plot_fft(nota.sintetizar3(44100)*15, elec_array, f_from = 100, f_to = 5000, range_dB = 60, sr = 44100, showElecs= True, show= False, norm = True, step =1)


Aca vemos como se alinean las freqs de analisis con las del sonido reasignado para ser procesado por el implante

In [ ]:
plot_fft(outCISin2, elec_array, f_from = 100, f_to = 5000, range_dB = 60, sr = elec_array.SR, show = False)
plt.vlines(elec_array.freqsAnalisis, -80, -60)
elec_array.freqsAnalisis

### Prueba de CIS nueva (con todos los addons y cambios de la tesis)

Lo siguiente nos muestra que el implante analiza bien las frecuencias que quería estimular

Mando el input reasignado que deberia estimular

Canales 2,4,5,6,8,9 con amps 1/n

In [ ]:
envs = CIS_6.filter_bank_analisis(outCISin2, elec_array, CIS_6.ORDEN) #el rippling se reduce con el orden de los filtros
envs = [envelope_Hilbert(envs[i]) for i in range(12)]
fig, axs = plt.subplots(1,4, figsize = (12,5), sharey= True)
for i, env in enumerate(envs):
    axs[i//3].plot(env, label = f'{i+1}', alpha = 0.8)
for ax in axs: ax.legend()

Amps = np.sum(envs, axis = 1)
print(np.round(Amps))

Veo si el CIS_ACE aplica bien las envolventes capturadas o si hace cualquier cosa

In [ ]:
us = [ [1]*len(envs[i]) for i in range(len(envs))] #aplico las envolventes a estos unit steps

outs = CIS_6.simulate_CIS_ACE2(envs, us, elec_array, 6, separate = True)

fig, axs = plt.subplots(1,4, figsize = (12,5), sharey= True)
for i, env in enumerate(outs):
    axs[i//3].plot(env, label = f'{i+1}', alpha = 0.85)
for ax in axs: ax.legend()

Amps = np.sum(envs, axis = 1)
print(np.round(Amps))

Lo hace bien. Elije unos canales, preserva sus amplitudes, y mata los otros

Mata canales 3, 7,9, 10,11,12 (mata el 9 porque ya eligió 6: 1,2,4,5,6,8)

Veamos ahora las amplitudes de las carriers que se le da a cada envolvente (los ruidos filtrados), a ver si tienen la misma amp:

In [ ]:
wnoise = wnoise_sig(len(envs[0]*10))
CIS_6.D = 0.005
filtered_noises = CIS_6.filter_bank_sintesis2(wnoise, elec_array)
out = np.sum(filtered_noises, axis = 0)
F = np.abs(fft(out))
F = F[0:len(out)//2]
plt.plot(F[0:5000])


El banco de filtros de outs no es lineal en amplitud, fijate... que raro. Pero esto debe ser consecuencia de como estoy sintetizando las señales en este metodo (ancho de banda de sintesis proporcional a los anchos de bandas criticos, que se agrandan con la frecuencia)

En dB esa diferencia no se nota

In [ ]:
wnoise = wnoise_sig(len(envs[0]*10))
filtered_noises = CIS_6.filter_bank_sintesis2(wnoise, elec_array)
out = np.sum(filtered_noises, axis = 0)
F = np.abs(fft(out))
F = F[0:len(out)//2]
plt.plot(20*np.log10(F[0:5000]))
plt.xlabel('n')

In [ ]:
wnoise = wnoise_sig(len(envs[0]))
filtered_noises = CIS_6.filter_bank_sintesis2(wnoise, elec_array)
out = np.sum(filtered_noises, axis = 0)
plot_fft(out, elec_array, f_from = 100, f_to = 5000, range_dB = 60, sr = CIS_6.OUT_SR, showElecs= True, show= False, logx = False)



Finalmente, hago el chequeo completo (recien hice paso a paso): Mando la señal reasignada y veo que solo se prenden esos canales que elijo

In [ ]:
CIS_6.DEBUG = True
CIS_6.D = 0.001
CIS_6.MAXIMA = 1
CIS_6.MODE = CIS_6.ACE
CIS_6.WIN_LEN_ACE = 400
outs = elec_array.CIS(outCISin2, input_sr= elec_array.SR)

fig, axs = plt.subplots(1,4, figsize = (12,5), sharey= True)
for i, env in enumerate(outs):
    axs[i//3].plot(env, label = f'{i+1}', alpha = 0.85)
for ax in axs: ax.legend()




Esto esta mal. No entiendo por que el canal 4 se selecciona antes que el 2... Eso si para mi no tiene sentido

Edit: Ya sé!! El Filtro inverso!!

Se lo agrego a las señales sintetizadas abajo y rehago las pruebas

In [ ]:
CIS_6.DEBUG = False
CIS_6.MAXIMA = 1
out = elec_array.CIS(outCISin2, input_sr= elec_array.SR)
plot_fft(out, elec_array, f_from = 100, f_to = 8000, range_dB = 60, sr = CIS_6.OUT_SR, showElecs= True, show= False, logx = True)

### Nuevo generador de ruido blanco

In [ ]:
#Genera una señal de ruido blanco de n samples
def wnoise_sig2(n):
    phases = np.random.rand(n)*2*np.pi
    return ifft(np.cos(phases) + 1j*np.sin(phases))
Audio(wnoise_sig2(44100) , rate = 44100)

In [ ]:
wnoise = wnoise_sig2(len(envs[0]*10))
CIS_6.D = 0.001
filtered_noises = CIS_6.filter_bank_sintesis2(wnoise, elec_array)
out = np.sum(filtered_noises, axis = 0)
F = np.abs(fft(out))
F = F[0:len(out)//2]
plt.plot(F[0:5000])


Cada vez que corro esto último, los picos suben y bajan random parece..

In [ ]:
wnoise = wnoise_sig2(len(envs[0]*10))
filtered_noises = CIS_6.filter_bank_sintesis2(wnoise, elec_array)
out = np.sum(filtered_noises, axis = 0)
F = np.abs(fft(out))
F = F[0:len(out)//2]
plt.plot(20*np.log10(F[0:5000]))
plt.xlabel('n')

### Reasignacion (sin compensar preenfasis)

In [ ]:
#origSin, origNoise, outOyenteSin, outOyenteNoise , outCISin2, outCINoise2 = elec_array.generarSenialesMelodia(melo_nota, sr = 44100)
plot_fft(origSin, elec_array, f_from = 100, f_to = 5000, range_dB = 60, sr = 44100)
plot_fft(origNoise, elec_array, f_from = 100, f_to = 5000, range_dB = 60, sr = 44100)
plot_fft(outOyenteSin, elec_array, f_from = 100, f_to = 5000, range_dB = 60, sr = 44100, showElecs= True, show=False)
plt.xlim(100)
plot_fft(outOyenteNoise, elec_array, f_from = 100, f_to = 5000, range_dB = 60, sr = 44100, showElecs= True, show= False)
plt.xlim(100)
plot_fft(out_reasignada, elec_array, f_from = 100, f_to = 5000, range_dB = 60, sr = CIS_6.OUT_SR, showElecs= True, show= False)
plt.xlim(100)

Todos los siguientes pares de scatter deberian ser iguales o casi iguales... dale

In [ ]:
ies = [i for i in range(len(nota.parciales))]
saw = [1/i for i in range(1,len(nota.parciales)+1)]
saw2 = [1/i**2 for i in range(1,len(nota.parciales)+1)]


FpruebaA = espectrosEspecificos(sig, nota.parciales, 44100, norm = True)
FpruebaA2 = espectrosEspecificos(origSin, nota.parciales, 44100, norm = True)
FpruebaB = espectrosEspecificos(origNoise, nota.parciales, 44100, norm = True)
FpruebaC = espectrosEspecificos(nota.sintetizar3(44100), nota.parciales, 44100, norm = True)


plt.scatter(ies, FpruebaA) #azul
plt.scatter(ies, FpruebaA2) #naranja
plt.scatter(ies, FpruebaB) #verde
plt.scatter(ies, FpruebaC) #rojo
plt.plot(ies, saw, linestyle = '--', color = 'purple')


In [ ]:
ies = [i for i in range(len(nota.parciales))]
saw = [1/i for i in range(1,len(nota.parciales)+1)]
saw2 = [1/i**2 for i in range(1,len(nota.parciales)+1)]


FpruebaA = espectrosEspecificos2(sig, nota.parciales, 44100, norm = True)
FpruebaA2 = espectrosEspecificos2(origSin, nota.parciales, 44100, norm = True)
FpruebaB = espectrosEspecificos2(origNoise, nota.parciales, 44100, norm = True)
FpruebaC = espectrosEspecificos2(nota.sintetizar3(44100), nota.parciales, 44100, norm = True)


plt.scatter(ies, FpruebaA) #azul
plt.scatter(ies, FpruebaA2) #naranja
plt.scatter(ies, FpruebaB) #verde
plt.scatter(ies, FpruebaC) #rojo
plt.plot(ies, saw, linestyle = '--', color = 'purple')


In [ ]:
ies = [i for i in range(12)]

Fprueba3 = espectrosEspecificos(outOyenteSin, elec_array.freqsElectrodos, 44100, norm = True)
Fprueba4 = espectrosEspecificos(outOyenteNoise, elec_array.freqsElectrodos, 44100, norm = True)
plt.scatter(ies, Fprueba3)
plt.scatter(ies, Fprueba4)

In [ ]:
ies = [i for i in range(12)]

Fprueba3 = espectrosEspecificos2(outOyenteSin, elec_array.freqsElectrodos, 44100, norm = True)
Fprueba4 = espectrosEspecificos2(outOyenteNoise, elec_array.freqsElectrodos, 44100, norm = True)
plt.scatter(ies, Fprueba3)
plt.scatter(ies, Fprueba4)

In [ ]:
ies = [i for i in range(12)]

Fprueba1 = espectrosEspecificos(outCISin2, elec_array.freqsAnalisis, elec_array.SR, norm = True)
Fprueba2 = espectrosEspecificos(outCINoise2, elec_array.freqsAnalisis, elec_array.SR, norm = True)

plt.scatter(ies, Fprueba1)
plt.scatter(ies, Fprueba2)

In [ ]:
ies = [i for i in range(12)]

Fprueba1 = espectrosEspecificos2(outCISin2, elec_array.freqsAnalisis, elec_array.SR, norm = True)
Fprueba2 = espectrosEspecificos2(outCINoise2, elec_array.freqsAnalisis, elec_array.SR, norm = True)

plt.scatter(ies, Fprueba1)
plt.scatter(ies, Fprueba2)

Y estos no, son los que quiero comparar (con vs sin reasignacion)

In [ ]:
ies = [i for i in range(12)]

F2 = espectrosEspecificos(out_normal, elec_array.freqsElectrodos, CIS_6.OUT_SR, norm = True)
F3 = espectrosEspecificos(out_reasignada, elec_array.freqsElectrodos, CIS_6.OUT_SR, norm  = True)
plt.scatter(ies, F2)
plt.scatter(ies, F3)

In [ ]:
ies = [i for i in range(12)]

F2 = espectrosEspecificos2(out_normal, elec_array.freqsElectrodos, CIS_6.OUT_SR, norm = True)
F3 = espectrosEspecificos2(out_reasignada, elec_array.freqsElectrodos, CIS_6.OUT_SR, norm  = True)
plt.scatter(ies, F2)
plt.scatter(ies, F3)

Ahi va... en conclusion, el metodo original que usaba (de sacar la DFT en puntos aislados) es MALISIMO para lo que queria hacer si la nota dura poco. Recien cuando empezo a durar 2 o mas segundos mejoro. Integrar la FFT en una region (acá usé fc +- 2.5Hz) funcionó infinitamente mejor para duraciones cortas.
Sin embargo, para las señales que quiero comparar (con vs sin reasignacion), ahi si empezó a funcionar peor...

### Reasignacion (compensando preenfasis)

In [ ]:
import import_ipynb
from CIS_6 import *
from CIS_6_Best_Match_v5 import * #notar el v5

In [ ]:
#Defino la nota que quiero
import pickle
with open('output una nota Medel.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    nota_puntuada_porclase = pickle.load(f)

actualizarInstanciaClasesN(nota_puntuada_porclase) #agrego el codigo actualizado de nota, porque la instancia se guardó con el codigo viejo



N = 6
puesto = 0

CIS_6.DEBUG = False

elec_array = GENERIC_MEDEL3
melo_nota = nota_puntuada_porclase[N][puesto]
melo_nota.printInformeScores()

nota = melo_nota.notas[0]
nota.vibDepth=0
nota.dur = 2 #hago que dure mas solo para tener mas puntos en las fft despues
melo_nota.dur = 2

sig = melo_nota.sintetizar(44100)


origSin, origNoise, outOyenteSin, outOyenteNoise , outCISin2, outCINoise2 = elec_array.generarSenialesMelodia(melo_nota, sr = 44100)
# 44100    44100        44100           44100          elecSR      elecSR
#sig == origSin

#genero la señal en los electrodos y desp analizo la energia en cada uno con y sin reasignacion
CIS_6.D = 0.02
out_normal = elec_array.CIS(sig, 44100)
out_reasignada = elec_array.CIS(outCISin2, elec_array.SR)

elec_array.generarTodasSeniales(melo_nota, sr = 44100)

In [ ]:
CIS_6.DEBUG = True
CIS_6.D = 0.001
CIS_6.MAXIMA = 1
CIS_6.MODE = CIS_6.ACE
CIS_6.WIN_LEN_ACE = 400
outs = elec_array.CIS(outCISin2, input_sr= elec_array.SR)

fig, axs = plt.subplots(1,4, figsize = (12,5), sharey= True)
for i, env in enumerate(outs):
    axs[i//3].plot(env, label = f'{i+1}', alpha = 0.85)
for ax in axs: ax.legend()

Aleluya!!! Anda!!

### Grafico

Por lo que dije arriba, voy a hacer la amplitud de los cuadrados (parciales) proporcional al dB, y no la amplitud, para reflejar todo esto

In [ ]:
def grafico_reasignacion(nota, normal, reasignada, xmin=250, xmax = 6500, elabelsize = 20):
    #####################################
    #Arriba: Electrodos activados (sin reasignación)
    #Medio:  Nota y sus armónicos
    #Abajo: Electrodos activados (con reasignación)

    #Para calcular las frecuencias máximas, voy a usar "find_peaks", sobre la fft

    ###########

    import matplotlib.ticker as ticker

    fes = elec_array.freqsElectrodos

    (y1, y2, y3) = (0.58,0.5, 0.42)

    #Generalidades gráfico
    fig, ax = plt.subplots(figsize=(16,6), dpi= 130)
    ax.set_xscale('log')
    ax.set_xlabel('Frecuencia (Hz)', labelpad = 50,fontdict={'size':25} )
    ax.xaxis.set_tick_params(pad = 7)
    ax.set_xlim(xmin , xmax)
    ax.set_ylim(0.35 , 0.8)
    ax.set_yticks([])
    ax.set_xticks([])
    ax.hlines(y=[y1, y2, y3], xmin=xmin, xmax=xmax, color='gray', alpha=0.85, linewidth=1, linestyles='dashdot')
    ax.vlines(x=fes, ymin=0, ymax=1, color='gray', alpha=0.6, linewidth=1.8, linestyles='dashdot')
    ax.vlines(x=nota.parciales, ymin=0, ymax=1, color='maroon', alpha=0.5, linewidth=1, linestyles='dashed')
    #ax.vlines(x=elec_array.freqsElectrodos, ymin=0.3, ymax=0.6, color='darkgray', alpha=0.95, linewidth=2, linestyles='dashdot') #Igual pero más corta y marcada

    #Title
    #ax.set_title('Simulación de electrodos estimulados con el esquema de reasignación', color = 'k', style = 'italic', pad = 20, fontdict={'size':32, 'font': 'Times New Roman'}, )

    #Texto para cada marquita de los elec
    elec_labels = ['$e_{' +str(i+1) +'}$ ' for i in range(elec_array.Nchann) ]

    #############################################3
    step = 1

    F1 = np.array(nota.amps)
    F2 = espectrosEspecificos2(normal, fes, CIS_6.OUT_SR, norm = True)
    F3 = espectrosEspecificos2(reasignada, fes, CIS_6.OUT_SR, norm  = True)

    #Paso a dB
    F2 = 20*np.log10(F2, where = (F2 != 0))
    F3 = 20*np.log10(F3, where = (F3 != 0))

    #"Normalizo" con el minimo ( a 0)
    #F1 -= min(F1)
    F2 -= min(F2)
    F3 -= min(F3)

    #Normalizo con el maximo (a 1)
    #F1 /= max(F1) /1000
    F2 /= max(F2)
    F3 /= max(F3)

    #a F2 y F3 hago mas drastico el contraste para visualizar mejor
    F1 *= 1000
    F2 = 1000*F2**10
    F3 = 1000*F3**10


    tuple_nota = (nota.parciales,F1, 'maroon', 'Componentes de la nota objetivo' , _, y1,'s')
    tuple_normal = (fes, F2, 'navy', 'Activaciones (sin reasignación)' , elec_labels, y2, 'D')
    tuple_realoc = (fes, F3, 'darkgreen', 'Activaciones (con reasignación)', elec_labels, y3, 'D')

    #Grafico las tres lineas de puntitos
    #El tamaño de los puntos es proporcional a la amplitud del espectro
    for frecuencias_amplitudes_color_label_ticklabels_y_marker in [tuple_nota, tuple_normal , tuple_realoc]:

        (freqs, amps, color, label, ticklabels, y0, forma) = frecuencias_amplitudes_color_label_ticklabels_y_marker #unpack

        #Grafico
        y = [ y0 for f in freqs]
        ax.scatter(y=y, x=freqs, s=np.array(amps), color=color, alpha=0.85, label = label, marker=forma) #Los puntitos
        ax.grid(alpha = 0.7)

    #Para los electodos apagados en la linea 3, repito más transparente la última linea y le agrego cositas:
    #ax.scatter(y=[0.3 for f in elec_array.freqsElectrodos], x=elec_array.freqsElectrodos, s=125, color='navy', alpha=0.45) #Los puntitos chiquitos y transparentes

    #Eje de abajo: e1, e2...
    secax = ax.secondary_xaxis('bottom') #eje secundario (texto)
    secax.set_xticks(fes[::step], labels = elec_labels[::step])
    secax.xaxis.set_tick_params(pad = 5, labelcolor = 'navy', labelsize = elabelsize*1.25, rotation = 30)
    for i in range(1,len(fes)//step+1):
        secax.get_xticklabels()[-i].set_rotation_mode('anchor')

    #Eje de abajo: Freqs electrodos
    secax_freqs = ax.secondary_xaxis('bottom') #eje secundario (numeros)
    freq_labels = [f'${round(freq)}$ ' for freq in fes ]
    secax_freqs.set_xticks(fes*1.025, labels = freq_labels[::step])
    secax_freqs.xaxis.set_tick_params(pad = 30, labelcolor = 'navy', labelsize= elabelsize)


    #Eje de arriba: freqs nota
    secax_freqs_nota = ax.secondary_xaxis('top') #eje secundario (numeros)
    freq_labels = [f'${round(freq)}$ ' for freq in nota.parciales ]
    secax_freqs_nota.set_xticks(nota.parciales, labels = freq_labels[::step])
    secax_freqs_nota.xaxis.set_tick_params(pad = 7, labelcolor = 'maroon', labelsize= 20)

    #ax.set_xticks([], minor=True)

    #Freqs de los armónicos
    for i, freq in enumerate(nota.parciales):
        plt.text(freq*0.975, y1*0.985, str(int(freq/nota.freq))+'°', fontdict={'size':20, 'font': 'Times New Roman', 'color':'yellow'})


    ######################################

    # Title, Label, Ticks and Ylim
    #ax.set_yticks(df.index)
    #ax.set_yticklabels(df.manufacturer.str.title(), fontdict={'horizontalalignment': 'right'})
    plt.legend(framealpha=1, shadow=True,  loc = 'upper left',  prop={'size': 24})
    plt.show()

In [ ]:
nota = melo_nota.notas[0]
grafico_reasignacion(nota, out_normal, out_reasignada, xmin =, xmax =)

Rehago pero las amps de la nota son [1,1,1,1,1]...

In [ ]:

nota.amps = [1,1,1,1,1,1]

sig = melo_nota.sintetizar(44100)


origSin, origNoise, outOyenteSin, outOyenteNoise , outCISin2, outCINoise2 = elec_array.generarSenialesMelodia(melo_nota, sr = 44100)
# 44100    44100        44100           44100          elecSR      elecSR
#sig == origSin

#genero la señal en los electrodos y desp analizo la energia en cada uno con y sin reasignacion
CIS_6.D = 0.02
CIS_6.DEBUG = False
CIS_6.MAXIMA = 8
out_normal = elec_array.CIS(sig, 44100)
out_reasignada = elec_array.CIS(outCISin2, elec_array.SR)

elec_array.generarTodasSeniales(melo_nota, sr = 44100)

In [ ]:
grafico_reasignacion(nota, out_normal, out_reasignada)

### Grafico 2 (para Cochlear)

In [ ]:
import pickle
with open('output una nota Cochlear Derecho.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    nota_puntuada_porclase = pickle.load(f)

actualizarInstanciaClasesN(nota_puntuada_porclase) #agrego el codigo actualizado de nota, porque la instancia se guardó con el codigo viejo

nota_puntuada_porclase[5][4].printInformeScores()
nota_puntuada_porclase[5][2].printInformeScores()
nota_puntuada_porclase[6][11].printInformeScores()

In [ ]:
N = 5
puesto = 4

elec_array = Cochlear_Axel_Der
melo_nota = nota_puntuada_porclase[N][puesto]
melo_nota.printInformeScores()

nota = melo_nota.notas[0]
nota.vibDepth=0
nota.dur = 2 #hago que dure mas solo para tener mas puntos en las fft despues
melo_nota.dur = 2

sig = melo_nota.sintetizar(44100)
nota.amps = [1]* N

origSin, origNoise, outOyenteSin, outOyenteNoise , outCISin2, outCINoise2 = elec_array.generarSenialesMelodia(melo_nota, sr = 44100)
# 44100    44100        44100           44100          elecSR      elecSR
#sig == origSin

#genero la señal en los electrodos y desp analizo la energia en cada uno con y sin reasignacion
CIS_6.D = 0.001
CIS_6.DEBUG = False
CIS_6.MAXIMA = 8
out_normal = elec_array.CIS(sig, 44100)
out_reasignada = elec_array.CIS(outCISin2, elec_array.SR)

#elec_array.generarTodasSeniales(melo_nota, sr = 44100)

In [ ]:
grafico_reasignacion(nota, out_normal, out_reasignada, xmin =1000, xmax = 7000, elabelsize = 15)

In [ ]:
len(melodias_puntuadas_porclase3[3])

In [ ]:
# Getting back the objects:
import pickle
with open('output seis notas Cochlear Derecho.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    melodias_puntuadas_porclase3 = pickle.load(f)
actualizarInstanciaClasesN(melodias_puntuadas_porclase3) #agrego el codigo actualizado de nota, porque la instancia se guardó con el codigo viejo


In [ ]:
melodias_puntuadas_porclase3[6][4].printInformeScores()

In [ ]:
# Getting back the objects:
import pickle
with open('output seis notas Cochlear Derecho.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    melodias_puntuadas_porclase3 = pickle.load(f)
actualizarInstanciaClasesN(melodias_puntuadas_porclase3) #agrego el codigo actualizado de nota, porque la instancia se guardó con el codigo viejo
